#Preliminaries

This is the proposed pipeline of the paper **Nisaba: Generating Natural Language Description of Multi-Perspective Declarative Process Models**. It contains the packages, utilities, prompts used for generation and the synthetic data generator using OpenAI API.

#Requirements

##Install dependencies

In [ ]:
!pip install declare4py==2.2.0
!pip install openai==1.35.7
!pip install pm4py==2.7.11.11
!pip install pyairports==2.1.1
!pip install pycountry==24.6.1
!pip install pydantic==2.7.4
!pip install pydantic_core==2.18.4
!pip install requests==2.31.0
!pip install requests-oauthlib==1.3.1
!pip install scipy==1.11.4
!pip install tenacity==8.4.2
!pip install termcolor==2.4.0
!pip install tiktoken==0.7.0
!pip install gradio==4.42.0

##Import dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import auth
from google.auth import default
import gspread
import pm4py
import Declare4Py as dec4py
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
import requests
from google.colab import userdata
from openai import OpenAI
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from enum import Enum
import os
from typing import List, Optional, Dict, Union
from pydantic import BaseModel, Field, conlist, conint, ValidationError, field_validator, model_validator
import re
import logging
import gradio as gr

##Utilities

This section is dedidated to set utilities to interplay with OpenAI API and set environment variables and model. It's necessary to have access to OpenAI key and have funds. You can set your OpenAI API key [here](https://platform.openai.com/settings/profile?tab=api-keys).

In [ ]:
# OpenAI environment variables configuration
GPT_MODEL = "gpt-4o-2024-05-13"
client = OpenAI(api_key=userdata.get('openai_key'))
os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, temperature, function_call=None, tools=None, tool_choice=None, model=GPT_MODEL):
    """
    Send a chat completion request to the GPT model and handle retries.

    This function sends a request to generate a chat completion based on the input messages and
    parameters. It automatically retries up to three times with exponential backoff in case of failure.

    Args:
        messages (list): A list of message dictionaries representing the conversation history.
        temperature (float): Sampling temperature to use. Higher values will make the output more random.
        function_call (dict, optional): A dictionary specifying a function call request. Defaults to None.
        tools (list, optional): A list of tools that the assistant can use. Defaults to None.
        tool_choice (str, optional): A specific tool choice to be used by the assistant. Defaults to None.
        model (str, optional): The GPT model to use for generating the completion. Defaults to `GPT_MODEL`.

    Returns:
        dict or Exception: Returns the response from the GPT model if successful, otherwise returns the exception.

    Raises:
        Exception: If unable to generate a ChatCompletion response after retries.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            messages=messages,
            tools=tools,
            function_call=function_call,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


def pretty_print_conversation(messages):
    """
    Print the conversation in a human-readable format with role-based color coding.

    This function takes a list of messages and prints each message with color coding based
    on the role of the sender (e.g., system, user, assistant, function).

    Args:
        messages (list): A list of message dictionaries representing the conversation history.

    Example:
        pretty_print_conversation([{'role': 'user', 'content': 'Hello!'}])

    """
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

# Extend class DeclareModel to add new methods to return attributes and binds.
class ExtendedDeclareModel(DeclareModel):
    """
    Extended version of the DeclareModel class from declare4py.

    This class adds two new methods to retrieve attributes and binds.
    """

    def get_decl_attributes(self):
        """
        Retrieve the list of attributes.

        This method iterates through the attributes list of the parsed model,
        collects attribute names and their values (if present), and returns a
        formatted list of attributes.

        Returns:
            list: A list of strings representing the attributes.
        """
        attributes = []
        for attr_name, attr_obj in self.parsed_model.attributes_list.items():
            attr_value = ""
            if attr_obj.attr_value:
                attr_value = ": " + attr_obj.attr_value.value_original
            attributes.append(f"{attr_name}{attr_value}")
        return attributes

    def get_decl_binds(self):
        """
        Retrieve the list of binds.

        This method iterates through the events in the parsed model, collects
        event names and their bound attributes, and returns a formatted list of
        binds.

        Returns:
            list: A list of strings representing the  binds.
        """
        binds = []
        for event_type, events in self.parsed_model.events.items():
            for event_name, event_obj in events.items():
                if event_obj.attributes:
                    bound_attrs = ", ".join(attr_obj.get_name() for attr_obj in event_obj.attributes.values())
                    binds.append(f"bind {event_name}: {bound_attrs}")
        return binds

# Configure the logger
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True,
    handlers=[
        logging.StreamHandler()
    ]
)

# Create a logger instance
logger = logging.getLogger('MyLogger')

###Prompts and Tools

In [ ]:
# The main prompt used to generate the intermediary descriptions.
decltotextassistant="""
You are an AI configured as an expert in declarative business process management, specifically for interpreting and describing Multi-Perspective Declare models (MP-Declare). Your primary task is to analyze the provided MP-Declare model and generate a comprehensive and consistent description. This description should meticulously reflect the structure and terminology of the model, maintaining exact naming and case sensitivity for model parts (activities, attributes, binds, and constraints). Examples of model descriptions are provided below (name of construct and example separated by ;) to guide your output and ensure alignment between data representation and text description. Your output will be the name of the construct and generated description.

# Example 1

- MP-Declare Model:

activity login
activity add_to_basket
activity remove_from_basket
activity confirmation
activity pay
activity get_discount
activity payment_success
activity order
activity shipping

bind login: user_id, password, confirm, user_type
bind add_to_basket: product, quantity, price
bind remove_from_basket: product, quantity
bind confirmation: confirm
bind pay: price, success, quantity, product
bind payment_success: confirm
bind order: status
bind shipping: shipped
bind get_discount: discount

user_id: integer between 1 and 100
password: float between 1000000 and 9000000
discount: integer between 5 and 15
quantity: integer between 1 and 50
product: cloths, radio, computer, notebook, food, chocolate
shipped: true, false
success: true, false
confirm: true, false
price: float between 0.99 and 5000.0
status: approved, cancelled, pending
user_type: admin, customer, delivery_boy


existence[login] | A.confirm is true |
Precedence[pay, confirmation] | | |

Response[pay, get_discount] |A.success is true and (A.price > 1000 or (A.product in (computer, notebook))) | T.discount = 10|
Response[pay, get_discount] |A.success is true and (A.price > 50 or (A.product in (chocolate, food))) | T.discount = 2|
Response[pay, get_discount] |A.price > 100 and (A.price < 1000) and A.success is true | T.discount = 5|
Response[pay, get_discount] | (A.success is false or A.quantity <= 0) and A.product not in (computer, notebook) or ((A.product is not food) and (A.product is not cloths)) | T.discount = 0|
Response[pay, get_discount] |A.price > 1500 and A.success is true | T.discount = 5|
Response[pay, order] |A.success is true | T.status is approved|
Response[pay, order] |A.success is false | T.status is cancelled|
Response[order, shipping] |A.status is approved | T.shipped is true|
Response[payment_success, get_discount] | | T.discount > 1|
Response[add_to_basket, pay] |  | |
Precedence[pay, login] | |T.user_type is customer|
Precedence[payment_success, pay] | |T.price >= 0.99|
Precedence[remove_from_basket, add_to_basket] | | |

- Description:

Activities:
- Name: login; Description: The 'login' activity represents the initial user authentication.
- Name: add_to_basket; Description: The 'add_to_basket' activity represents adding items to the shopping basket.
- Name: remove_from_basket; Description: The 'remove_from_basket' activity represents removing items from the shopping basket.
- Name: confirmation; Description: The 'confirmation' activity represents confirming order or payment details.
- Name: pay; Description: The 'pay' activity represents processing payment for items.
- Name: get_discount; Description: The 'get_discount' activity represents applying discounts based on specific conditions.
- Name: payment_success; Description: The 'payment_success' activity represents successful payment processing.
- Name: order; Description: The 'order' activity represents managing the status of an order.
- Name: shipping; Description: The 'shipping' activity represents dispatching the order to the customer.


Attributes:
- Name: user_id: Integer between 1 and 100; Description: The attribute 'user_id: integer between 1 and 100' represents the user's identification number for authentication purposes.
- Name: password: Float between 1000000 and 9000000; Description: The attribute 'password: float between 1000000 and 9000000' represents the user's password for login.
- Name: product: Cloths, radio, computer, notebook, food, chocolate; Description: The attribute 'product: cloths, radio, computer, notebook, food, chocolate' specifies the types of products available for purchase.
- Name: price: Float between 0.99 and 5000.0; Description: The attribute 'price: float between 0.99 and 5000.0' indicates the price range of the products, from 0.99 to 5000 dollars.
- Name: discount: Integer between 5 and 15; Description: The attribute 'discount: integer between 5 and 15' denotes the range of discount that can be applied to a transaction.
- Name: quantity: Integer between 1 and 50; Description: The attribute 'quantity: integer between 1 and 50' represents the number of units of a product a user is purchasing.
- Name: shipped: True, False; Description: The attribute 'shipped: true, false' indicates whether the goods have been shipped.
- Name: success: True, False; Description: The attribute 'success: true, false' indicates the outcome of a transaction which can be True or False.
- Name: confirm: True, False; Description: The attribute 'confirm: true, false' indicates whether the transaction or delivery has been confirmed by the user and can be True or False.
- Name: user_type: Admin, customer, delivery_boy; Description: The attribute 'user_type: admin, customer, delivery_boy' describes the role of the user within the system and can be admin, customer or delivery_boy.
- Name: status: approved, cancelled, pending; Description: The attribute 'status: approved, cancelled, pending' indicates wheter the order is status with possibles values being approved, cancelled or pendind.


Binds:
- Name: login: user_id, password, confirm, user_type; Description: This bind connects the 'login' activity with the attributes 'user_id', 'password', 'confirm', and 'user_type', ensuring that user authentication is tied to these specific credentials and roles.
- Name: add_to_basket: product, quantity, price; Description: This bind links the 'add_to_basket' activity with the attributes 'product', 'quantity', and 'price', associating the addition of items to the basket with their specific types, quantities, and prices.
- Name: remove_from_basket: product, quantity; Description: This bind associates the 'remove_from_basket' activity with the attributes 'product' and 'quantity', linking the removal of items from the basket with the specific product and the quantity removed.
- Name: confirmation: confirm; Description: This bind ties the 'confirmation' activity to the 'confirm' attribute, indicating that confirmation is dependent on this boolean status.
- Name: pay: price, success, quantity, product; Description: This bind relates the 'pay' activity to the attributes 'price', 'success', 'quantity', and 'product', connecting payment processing to these specific details of the transaction.
- Name: payment_success: confirm; Description: This bind connects the 'payment_success' activity to the 'confirm' attribute, indicating that successful payment acknowledgment is tied to the confirmation status.
- Name: order: status; Description: This bind associates the 'order' activity with the 'status' attribute, linking order management to the current status of the order (approved, cancelled, pending).
- Name: shipping: shipped; Description: This bind links the 'shipping' activity to the 'shipped' attribute, indicating that the shipping process is dependent on whether the order has been dispatched.
- Name: get_discount: discount; Description: This bind couples the 'get_discount' activity to the 'discount' attribute, associating the application of discounts with the amount awarded.


Constraints:
- Name: existence[login] | A.confirm is true |; Description: This constraint ensures that the 'login' activity must exist if the attribute 'confirm' is true. It guarantees that a login is required whenever confirmation is provided.
- Name: Precedence[pay, confirmation] | | |; Description: This constraint specifies that the 'pay' activity can only occur after the 'confirmation' activity, ensuring that payment is processed only after order confirmation.
- Name: Response[pay, get_discount] | A.success is true and (A.price > 1000 or A.product in (computer, notebook)) | T.discount = 10 |; Description: This constraint applies a 10% discount after the 'pay' activity if the payment is successful and the 'price' is over $1000 or the 'product' is a computer or notebook, linking payment success and product type to the discount.
- Name: Response[pay, get_discount] | A.success is true and (A.price > 50 or A.product in (chocolate, food)) | T.discount = 2 |; Description: This constraint applies a 2% discount after the 'pay' activity if the payment is successful and the 'price' is over $50 or the 'product' is chocolate or food, linking payment success and product type to the discount.
- Name: Response[pay, get_discount] | A.price > 100 and A.price < 1000 and A.success is true | T.discount = 5 |; Description: This constraint grants a 5% discount after the 'pay' activity if the payment is successful and the 'price' is between $100 and $1000, establishing an intermediate price range for the discount.
- Name: Response[pay, get_discount] | (A.success is false or A.quantity <= 0) and A.product not in (computer, notebook) and A.product not in (food, cloths) | T.discount = 0 |; Description: This constraint ensures no discount is applied after the 'pay' activity if the payment fails, the 'quantity' is zero or negative, or the 'product' is not a computer, notebook, food, or clothes, preventing discounts in invalid situations.
- Name: Response[pay, get_discount] | A.price > 1500 and A.success is true | T.discount = 5 |; Description: This constraint awards a 5% discount after the 'pay' activity if the payment is successful and the 'price' is over $1500, promoting discounts for high-value transactions.
- Name: Response[pay, order] | A.success is true | T.status = approved |; Description: This constraint changes the 'order' status to 'approved' after the 'pay' activity if the payment is successful, linking payment success directly to order approval.
- Name: Response[pay, order] | A.success is false | T.status = cancelled |; Description: This constraint changes the 'order' status to 'cancelled' after the 'pay' activity if the payment fails, linking payment failure to order cancellation.
- Name: Response[order, shipping] | A.status = approved | T.shipped = true |; Description: This constraint ensures the 'shipping' activity occurs after the 'order' activity if the order status is 'approved', ensuring only approved orders are shipped.
- Name: Response[payment_success, get_discount] | | T.discount > 1 |; Description: This constraint applies a discount greater than 1% after the 'payment_success' activity, ensuring a discount is applied following successful payment.
- Name: Response[add_to_basket, pay] | | |; Description: This constraint requires the 'pay' activity to follow the 'add_to_basket' activity, establishing a logical sequence from adding items to the basket to making a payment.
- Name: Precedence[pay, login] | | T.user_type = customer |; Description: This constraint ensures the 'pay' activity follows the 'login' activity, confirming the user is a customer, and allowing only customers to make payments.
- Name: Precedence[payment_success, pay] | | T.price >= 0.99 |; Description: This constraint ensures the 'payment_success' activity follows the 'pay' activity, with a minimum 'price' of $0.99, setting a minimum transaction value for payment success.
- Name: Precedence[remove_from_basket, add_to_basket] | | |; Description: This constraint requires the 'remove_from_basket' activity to follow the 'add_to_basket' activity, establishing the correct order for managing the shopping basket.

# Example 2
- MP-Declare Model:
activity Apply for Trip
activity Approve application
activity Book means of transport
activity Book accommodation
activity Collect tickets
activity Archive documents


bind Apply for Trip: Application
bind Collect tickets: TicketCollection
bind Book means of transport: org::resource
bind Book accommodation: org::resource
bind Collect tickets: org::resource
bind Archive documents: org::resource
bind Apply for Trip: org::resource

Application: Application.written
TicketCollection: TicketCollection.written
org::resource: Alice, Bob


Precedence[Approve application, Apply for Trip B]||B.Application is Application.written
Precedence[Archive documents, Apply for Trip B]||B.Application is Application.written
Precedence[Archive documents, Collect tickets B]||B.TicketCollection is TicketCollection.written

Precedence[Book means of transport, Approve application]
Precedence[Book accommodation, Approve application]
Precedence[Collect tickets, Book means of transport]
Precedence[Collect tickets, Book accommodation]

Existence[Apply for Trip]
Existence[Approve application]
Existence[Collect tickets]
Existence[Archive documents]

- Description:

Activities:
- Name: Apply for Trip; Description: The 'Apply for Trip' activity represents the initial action where an individual submits an application for a trip.
- Name: Approve application; Description: The 'Approve application' activity involves the review and approval of the trip application.
- Name: Book means of transport; Description: The 'Book means of transport' activity entails making arrangements for transportation for the trip.
- Name: Book accommodation; Description: The 'Book accommodation' activity involves securing lodging for the duration of the trip.
- Name: Collect tickets; Description: The 'Collect tickets' activity represents the action of collecting tickets for transport and possibly for accommodations or events.
- Name: Archive documents; Description: The 'Archive documents' activity involves the process of organizing and storing documents related to the trip, such as receipts, tickets, and approvals, after the trip has concluded.

Attributes:
- Name: Application: Application.written; Description: The attribute 'Application: Application.written' indicates whether the trip application has been formally documented and submitted with the status 'written'.
- Name: TicketCollection: TicketCollection.written; Description: The attribute 'TicketCollection: TicketCollection.written' indicates the status 'written', focusing on whether the process of collecting tickets has been documented.
- Name: org::resource: Alice, Bob; Description: The attribute 'org::resource: Alice, Bob' represents the organizational resources involved in the trip planning process, which includes individuals named Alice and Bob.

Binds:
- Name: Apply for Trip: Application; Description: This bind links the 'Apply for Trip' activity with the attribute 'Application.written', indicating that the activity relies on the status of the application being formally documented and submitted.
- Name: Apply for Trip: org::resource; Description: This bind connects the 'Apply for Trip' activity with the organizational resources 'Alice, Bob', specifying that these individuals are involved in the application process.
- Name: Collect tickets: TicketCollection; Description: This bind associates the 'Collect tickets' activity with the attribute 'TicketCollection.written', highlighting that the collection process is documented.
- Name: Collect tickets: org::resource; Description: This bind links the 'Collect tickets' activity with the organizational resources 'Alice, Bob', indicating their involvement in the ticket collection process.
- Name: Book means of transport: org::resource; Description: This bind connects the 'Book means of transport' activity with the attribute 'org::resource' which represents organizational resources 'Alice, Bob', suggesting their responsibility in arranging transportation.
- Name: Book accommodation: org::resource; Description: This bind links the 'Book accommodation' activity with the organizational resources 'Alice, Bob', implying that they are responsible for securing lodging.
- Name: Archive documents: org::resource; Description: This bind associates the 'Archive documents' activity with the organizational resources 'Alice, Bob', indicating their role in organizing and storing trip-related documents.

Constraints:
- Name: Precedence[Approve application, Apply for Trip B]||B.Application is Application.written; Description: This constraint specifies that the 'Approve application' activity must follow the 'Apply for Trip' activity, provided the trip application is in a 'written' status, ensuring that only documented applications are processed.
- Name: Precedence[Archive documents, Apply for Trip B]||B.Application is Application.written; Description: Indicates that the 'Archive documents' activity can only occur after the 'Apply for Trip' activity if the trip application has been submitted, ensuring proper documentation flow.
- Name: Precedence[Archive documents, Collect tickets B]||B.TicketCollection is TicketCollection.written; Description: Suggests that the 'Archive documents' activity must follow the 'Collect tickets' activity once ticket collection is documented, maintaining order in document processing.
- Name: Precedence[Book means of transport, Approve application]; Description: Ensures that the 'Book means of transport' activity can only be happen after the 'Approve application' activity, aligning travel arrangements with approved plans.
- Name: Precedence[Book accommodation, Approve application]; Description: Ensures that the 'Book accomodation' activity can only happen after the trip 'Approve application', coordinating lodging with approved travel plans.
- Name: Precedence[Collect tickets, Book means of transport]; Description: Ensures that 'Collect tickets' activity follows the 'Book means of transport' activity, streamlining the travel preparation process.
- Name: Precedence[Collect tickets, Book accommodation]; Description: Ensures that 'Collect tickets'follows the 'Book accomodation', facilitating smooth travel arrangements.

- Name: Existence[Apply for Trip]; Description: Ensures that the 'Apply for Trip' activity must occur at least once in the process, initiating the sequence of travel-related activities.
- Name: Existence[Approve application]; Description: Guarantees that the 'Approve application' activity must occur at least once in the process, ensuring all trips are duly authorized.
- Name: Existence[Collect tickets]; Description: Ensures that the 'Collect tickets' activity occurs at least once in the process, securing all necessary tickets for the trip.
- Name: Existence[Archive documents]; Description: Guarantees that the 'Archive documents' activity occurs at least once in the process, ensuring all documents are properly archived post-trip.

# Example 3
- MP-Declare Model:
activity Driving_Test
activity Getting_License
activity Resit
activity Test_Failed

bind Driving_Test: Driver, Grade
bind Getting_License: Driver, Grade
bind Resit: Driver, Grade
bind Test_Failed: Driver

Driver: Fabrizio, Mike, Marlon, Raimundas
Grade: integer between 1 and 5

Response[Driving_Test, Getting_License] | |T.Grade>2 |
Response[Driving_Test, Resit] |A.Grade<=2 | |
Response[Driving_Test, Test_Failed] |A.Grade<=2 | |

- Description:

Activities:
- Name: Driving_Test; Description: The 'Driving_Test' activity represents the initial driving test taken by a candidate.
- Name: Getting_License; Description: The 'Getting_License' activity signifies the action of receiving a driving license upon passing the driving test.
- Name: Resit; Description: The 'Resit' activity represents the action of retaking the driving test after failing.
- Name: Test_Failed; Description: The 'Test_Failed' activity represents the outcome of failing the driving test.


Attributes:
- Name: Driver: Fabrizio, Mike, Marlon, Raimundas; Description: The attribute 'Driver: Fabrizio, Mike, Marlon, Raimundas' represents an individual eligible to participate in the driving test process with possible values being Fabrizio, Mike, Marlon, and Raimundas.
- Name: Grade: integer between 1 and 5; Description: The attribute 'Grade: integer between 1 and 5' is defined as an integer between 1 and 5. It quantifies the performance of a candidate in the driving test, influencing their eligibility for subsequent activities like 'Getting_License' or 'Resit'.

Binds:
- Name: Driving_Test: Driver, Grade; Description: This bind specifies that the 'Driving_Test' activity requires the attributes 'Driver' and 'Grade'. It indicates that each instance of this activity is associated with a specific driver and their corresponding performance grade.
- Name: Getting_License: Driver, Grade; Description: This bind connects the 'Getting_License' activity with the 'Driver' and the 'Grade' they received. It implies that the activity is contingent upon receiving a passing grade, indicating successful completion of the 'Driving_Test'.
- Name: Resit: Driver, Grade; Description: This bind associates the 'Resit' activity with both 'Driver' and 'Grade'. It reflects the need for a driver who failed the initial test to retake it, dependent on their previous grade.
- Name: Test_Failed: Driver; Description: This bind indicates that the 'Test_Failed' activity is solely linked to the 'Driver', without direct consideration of the 'Grade'. It signifies that the activity focuses on the occurrence of failure rather than the specific grade received.

Constraints:
- Name: Response[Driving_Test, Getting_License] | |T.Grade>2 |; Description: This constraint requires that after completing the 'Driving_Test', the 'Getting_License' activity must occur if the candidate's Grade is above 2. It explicitly states that a minimum grade of 3 is necessary to qualify for a license, linking performance directly to licensing outcomes.
- Name: Response[Driving_Test, Resit] |A.Grade<=2 | |; Description: This constraint mandates that candidates who score 2 or below on the 'Driving_Test' must retake the test through the 'Resit' activity. It addresses the potential for candidates to improve and successfully pass on a subsequent attempt.
- Name: Response[Driving_Test, Test_Failed] |A.Grade<=2 | |; Description: This constraint activates the 'Test_Failed' activity if a candidate's grade on the 'Driving_Test' is 2 or below. It formally recognizes and records the failure, potentially leading to a retake of the test.
"""

# 4 functions to get model description.
toolsdescription = [
  {
    "type": "function",
    "function": {
      "name": "get_activities_description",
      "description": "Fetch the activities of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "activities": {
            "type": "array",
            "description": "The activities contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the activity."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the activity."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_attributes_description",
      "description": "Fetch the attributes of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "attributes": {
            "type": "array",
            "description": "The attributes contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the attribute."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the attribute."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_binds_description",
      "description": "Fetch the binds of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "binds": {
            "type": "array",
            "description": "The binds contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the bind."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the bind."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_constraints_description",
      "description": "Fetch the constraints of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "constraints": {
            "type": "array",
            "description": "The constraints contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the constraint."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the constraint."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  }
]

enhdescassistant ="""
# AI Business Process Interpretation Prompt

You are an AI trained to interpret and describe business processes based on declarative process models. Your task is to generate a continuous and detailed description of business processes from a set of provided activities, attributes, binds, and constraints. Follow this strategy when describing the process:

## 1. Overall Activity Context

- Describe the general purpose and scope of the process.
- Explain how activities are generally related to each other.

## 2. Process Flow Description

Begin with the process initiation and progress through each activity, ensuring a logical flow that reflects the constraints and relationships defined in the model. For each activity:

- Explain how it connects to previous and subsequent activities.
- Highlight attributes influencing its execution.
- Detail how specific constraints control the flow to and from this activity.
- Consider dependencies, order of execution, and the impact of outcomes from previous activities.

## 3. Constraint Integration

Throughout the description, incorporate the following constraints only if they apply to activities in the original model:

### Unary Constraints

- **Existence**: Specify when an activity must occur.
- **Absence**: Indicate when an activity must not occur.
- **Exactly**: Describe when an activity must occur a specific number of times.
- **Init**: Identify activities that can or must start the process.
- **End**: Point out activities that can or must end the process.

### Binary Constraints

- **Choice and Exclusive Choice**: Explain options between activities.
- **Responded Existence**: Describe when the occurrence of one activity requires another.
- **Response, Alternate Response, and Chain Response**: Detail how activities trigger others.
- **Precedence, Alternate Precedence, and Chain Precedence**: Clarify required order of activities.
- **Succession, Alternate Succession, and Chain Succession**: Explain bidirectional dependencies.
- **Co-Existence**: Highlight activities that must occur together.
- **Not Chain Succession, Not Co-Existence, Not Succession**: Describe prohibited sequences or combinations.
- **Not Responded Existence, Not Response, Not Precedence**: Explain when certain follow-ups are forbidden.
- **Not Chain Response, Not Chain Precedence**: Detail prohibited immediate sequences.

## 4. Constraint Application

For each constraint applied:

- Explain its impact on the process flow.
- Describe how it affects decision-making or branching in the process.
- Clarify any conditions or exceptions related to the constraint.

## 5. Conclusion

Conclude by summarizing the key aspects of the process, highlighting the most critical constraints and their overall impact on the business process execution.

---

**Note**: Maintain a logical flow in your description, connecting activities and constraints in a way that reflects real-world process execution. Your explanation should be detailed yet comprehensible, suitable for both process experts and those new to the specific business domain.\n"""

#Application

In [ ]:
declare_model = ExtendedDeclareModel()
declare_model.parse_from_string(""""
asdasdas
""")

In [ ]:
declare_model.declare_model_lines

In [ ]:
def handle_input(input_type, text_input, file_input):
    declare_model = ExtendedDeclareModel()
    if input_type == "Lines":
        model = text_input
    elif input_type == "File" and file_input is not None:
        try:
            with open(file_input.name, 'r', encoding='utf-8') as f:
                model = f.read()
        except Exception as e:
            return f"Error reading file: {str(e)}"
    else:
        return "Please provide valid input."

    try:
        declare_model.parse_from_string(model)
        declare_model.declare_model_lines
        return "Model parsed successfully."
    except Exception as e:
        return f"Failed to parse model: {str(e)}"

def show_additional_outputs(output):
    if "Model parsed successfully." in output:
        return gr.update(visible=True), gr.update(visible=True)
    else:
        return gr.update(visible=False), gr.update(visible=False)

with gr.Blocks() as demo:
    gr.Markdown("# MP-Declare Model Description Generator")

    with gr.Row():
        input_type = gr.Radio(choices=["Lines", "File"], label="Choose Import Type")

    text_input = gr.Textbox(label="Input Model Lines", visible=False)
    file_input = gr.File(label="Upload .decl File", visible=False)

    def update_ui(choice):
        if choice == "Lines":
            return gr.update(visible=True), gr.update(visible=False)
        elif choice == "File":
            return gr.update(visible=False), gr.update(visible=True)

    input_type.change(update_ui, input_type, [text_input, file_input])

    submit_btn = gr.Button("Submit")

    # Make the output textbox non-editable by setting interactive=False
    output = gr.Textbox(label="Output", interactive=False)

    # New textboxes for intermediary and continuous descriptions, initially hidden
    intermediary_descriptions = gr.Textbox(label="Intermediary Descriptions", visible=False)
    continuous_descriptions = gr.Textbox(label="Continuous Descriptions", visible=False)

    # First, handle input and display the output
    submit_btn.click(handle_input, [input_type, text_input, file_input], output)

    # Then, based on the output, control the visibility of the additional textboxes
    output.change(show_additional_outputs, output, [intermediary_descriptions, continuous_descriptions])

demo.launch()


In [ ]:
def format_prompt(declare_model_lines):
    model = "\n".join(declare_model_lines)
    prompt = "New model to generate description of activities, attributes, binds, and constraints based on the template: \n" + model
    return prompt

def process_intermediate_descriptions(tool_calls):
    model = {"Activities": [], "Attributes": [], "Binds": [], "Constraints": []}
    try:
        for item in tool_calls:
            func_args = json.loads(item.function.arguments)
            for key, value in func_args.items():
                for v in value:
                    model[key.capitalize()].append(f"{v['name']}: {v['description']}")
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error processing descriptions: {e}")
        return ""

    model_string = ""
    for key, descriptions in model.items():
        model_string += f"{key}:\n"
        for desc in descriptions:
            model_string += f"  - {desc}\n"

    return model_string

def generate_intermediary_description(declare_model_lines, decltotextassistant, toolsdescription):
    model_prompt = format_prompt(declare_model_lines)
    messagesmptodesc = [
        {"role": "system", "content": decltotextassistant},
        {"role": "user", "content": model_prompt}
    ]

    response = chat_completion_request(
        messages=messagesmptodesc,
        tools=toolsdescription,
        tool_choice="auto",
        temperature=0.2
    )

    if response:
        description_string = process_intermediate_descriptions(response.choices[0].message.tool_calls)
        return description_string
    else:
        return "Failed to generate description."

def handle_input(input_type, text_input, file_input, decltotextassistant, toolsdescription):
    declare_model = ExtendedDeclareModel()
    if input_type == "Lines":
        model = text_input
    elif input_type == "File" and file_input is not None:
        try:
            with open(file_input.name, 'r', encoding='utf-8') as f:
                model = f.read()
        except Exception as e:
            return f"Error reading file: {str(e)}", "", ""
    else:
        return "Please provide valid input.", "", ""

    try:
        declare_model.parse_from_string(model)
        declare_model_lines = declare_model.declare_model_lines

        intermediary_description = generate_intermediary_description(declare_model_lines, decltotextassistant, toolsdescription)

        return "Model parsed successfully.", intermediary_description, ""
    except Exception as e:
        return f"Failed to parse model: {str(e)}", "", ""

def show_additional_outputs(output):
    if "Model parsed successfully." in output:
        return gr.update(visible=True), gr.update(visible=True)  # Show both textboxes
    else:
        return gr.update(visible=False), gr.update(visible=False)

# Wrapper function that includes the external arguments
def handle_input_wrapper(input_type, text_input, file_input):
    return handle_input(input_type, text_input, file_input, decltotextassistant, toolsdescription)

with gr.Blocks() as demo:
    gr.Markdown("# MP-Declare Model Description Generator")

    with gr.Row():
        input_type = gr.Radio(choices=["Lines", "File"], label="Choose Import Type")
    text_input = gr.Textbox(label="Input Model Lines", visible=False)
    file_input = gr.File(label="Upload .decl File", visible=False)

    def update_ui(choice):
        if choice == "Lines":
            return gr.update(visible=True), gr.update(visible=False)
        elif choice == "File":
            return gr.update(visible=False), gr.update(visible=True)

    input_type.change(update_ui, input_type, [text_input, file_input])

    submit_btn = gr.Button("Submit")

    output = gr.Textbox(label="Output", interactive=False, lines=2)
    intermediary_descriptions = gr.Textbox(
        label="Intermediary Descriptions",
        visible=False,
        interactive=True,  # Make it adjustable
        lines=10  # Set initial height
    )
    final_descriptions = gr.Textbox(
        label="Final Descriptions",
        visible=False,
        interactive=True,  # Make it adjustable
        lines=10  # Set initial height
    )

    # Handle input and display the output and intermediary descriptions using the wrapper function
    submit_btn.click(fn=handle_input_wrapper, inputs=[input_type, text_input, file_input], outputs=[output, intermediary_descriptions])

    # Control visibility of both intermediary and final descriptions
    output.change(show_additional_outputs, output, [intermediary_descriptions, final_descriptions])

demo.launch()


In [ ]:
def generate_final_descriptions(description_string, enhdescassistant):
    messagesendesc = [
        {"role": "system", "content": enhdescassistant},
        {"role": "user", "content": "New model to generate description: " + description_string}
    ]

    responseendesc = chat_completion_request(
        messages=messagesendesc,
        temperature=0.2
    )

    if responseendesc:
        final_description = responseendesc.choices[0].message.content
        return final_description
    else:
        return "Failed to generate enhanced description."

def handle_input(input_type, text_input, file_input, decltotextassistant, toolsdescription, enhdescassistant):
    declare_model = ExtendedDeclareModel()
    if input_type == "Lines":
        model = text_input
    elif input_type == "File" and file_input is not None:
        try:
            with open(file_input.name, 'r', encoding='utf-8') as f:
                model = f.read()
        except Exception as e:
            return f"Error reading file: {str(e)}", "", ""
    else:
        return "Please provide valid input.", "", ""

    try:
        declare_model.parse_from_string(model)
        declare_model_lines = declare_model.declare_model_lines

        intermediary_description = generate_intermediary_description(declare_model_lines, decltotextassistant, toolsdescription)
        final_description = generate_final_descriptions(intermediary_description, enhdescassistant)

        return "Model parsed successfully.", intermediary_description, final_description
    except Exception as e:
        return f"Failed to parse model: {str(e)}", "", ""

def show_additional_outputs(output):
    if "Model parsed successfully." in output:
        return gr.update(visible=True), gr.update(visible=True)  # Show both textboxes
    else:
        return gr.update(visible=False), gr.update(visible=False)

# Wrapper function that includes the external arguments
def handle_input_wrapper(input_type, text_input, file_input):
    return handle_input(input_type, text_input, file_input, decltotextassistant, toolsdescription, enhdescassistant)

with gr.Blocks() as demo:
    gr.Markdown("# MP-Declare Model Description Generator")

    with gr.Row():
        input_type = gr.Radio(choices=["Lines", "File"], label="Choose Import Type")
    text_input = gr.Textbox(label="Input Model Lines", visible=False)
    file_input = gr.File(label="Upload .decl File", visible=False)

    def update_ui(choice):
        if choice == "Lines":
            return gr.update(visible=True), gr.update(visible=False)
        elif choice == "File":
            return gr.update(visible=False), gr.update(visible=True)

    input_type.change(update_ui, input_type, [text_input, file_input])

    submit_btn = gr.Button("Submit")

    output = gr.Textbox(label="Output", interactive=False, lines=1)
    intermediary_descriptions = gr.Textbox(
        label="Intermediary Descriptions",
        visible=False,
        interactive=False,  # Make it adjustable
        lines=10  # Set initial height
    )
    final_descriptions = gr.Textbox(
        label="Final Descriptions",
        visible=False,  # This ensures it only shows after submit
        interactive=False,  # Still allow editing if necessary
        lines=10  # Set initial height
    )

    # Handle input and display the output and intermediary descriptions using the wrapper function
    submit_btn.click(fn=handle_input_wrapper, inputs=[input_type, text_input, file_input], outputs=[output, intermediary_descriptions, final_descriptions])

    # Control visibility of both intermediary and final descriptions
    output.change(show_additional_outputs, output, [intermediary_descriptions, final_descriptions])

demo.launch()


#Tool Evaluation

##MP-Declare Data Model

The following data model were used to validate if MP-Declare models are compliant with Declare4Py specifications.

In [ ]:
# Activity Model
class Activity(BaseModel):
    """
    Represents an activity in a process model.

    Attributes:
        name (str): The name of the activity, following the format '<Action Verb> <Object>' or '<Object> <Action Verb>'.
        description (str): A detailed description of what the activity entails.
    """
    name: str = Field(
        ...,
        description="The name of the activity. Should follow the format '<Action Verb> <Object>' or '<Object> <Action Verb>'.",
        #pattern=r"^([A-Z][a-z]*\s+[A-Z][a-z]*)(\s+[A-Z][a-z]*){0,4}$",
        min_length=1
    )
    description: str = Field(
        ...,
        description="A detailed description of what the activity entails.",
        min_length=1
    )

# Attribute Model
class Attribute(BaseModel):
    """
    Represents an attribute associated with an activity.

    Attributes:
        type (str): The type of attribute. It can be integer, float or enumeration.
        name (str): The name of the attribute.
        description (str): A detailed description of what the attribute represents.
        min_value (Optional[Union[float, int]]): Minimum value for integer or float attributes.
        max_value (Optional[Union[float, int]]): Maximum value for integer or float attributes.
        enumeration_values (Optional[List[str]]): Possible values for enumeration attributes.
    """
    type: str = Field(
        ...,
        description="The type of attribute. It can be integer, float or enumeration",
        pattern=r"^(integer|float|enumeration)$"
    )
    name: str = Field(
        ...,
        description="The name of the attribute.",
        min_length=1
    )
    description: str = Field(
        ...,
        description="A detailed description of what the attribute represents.",
        min_length=1
    )
    min_value: Optional[Union[float, int]] = Field(
        None,
        description="Minimum value for integer or float attributes. Should be <= than max_value"
    )
    max_value: Optional[Union[float, int]] = Field(
        None,
        description="Maximum value for integer or float attributes. Should be >= than min_value"
    )
    enumeration_values: Optional[List[str]] = Field(
        None,
        description="Possible values for enumeration attributes. Should be more than 1"
    )

    @field_validator('min_value', 'max_value', mode='before')
    def check_min_max_values(cls, value, info):
        """
        Validate min_value and max_value based on the type of attribute.

        Raises:
            ValueError: If min_value or max_value are required but not provided.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Validating {info.field_name} for attribute type {info.data['type']}.")
        if info.data['type'] == 'integer':
            if value is not None:
                value = int(value)
            else:
                raise ValueError(f"{info.field_name} is required for {info.data['type']} type.")
        elif info.data['type'] == 'float':
            if value is None:
                raise ValueError(f"{info.field_name} is required for {info.data['type']} type.")
        return value

    @field_validator('min_value', 'max_value', mode='after')
    def ensure_min_less_than_max(cls, value, info):
        """
        Ensure that min_value is less than or equal to max_value.

        Raises:
            ValueError: If min_value is greater than max_value.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Ensuring {info.field_name} for attribute type {info.data['type']} is within valid range.")
        if info.data.get('min_value') is not None and info.data.get('max_value') is not None:
            if info.data['min_value'] > info.data['max_value']:
                raise ValueError("min_value must be less than or equal to max_value.")
        return value

    @field_validator('enumeration_values', mode='before')
    def check_enumeration_values(cls, value, info):
        """
        Validate enumeration_values for enumeration type attributes.

        Raises:
            ValueError: If enumeration_values has fewer than 2 unique items.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Validating enumeration values for attribute type {info.data['type']}.")
        if info.data['type'] == 'enumeration':
            if not value or len(value) < 2:
                raise ValueError("enumeration_values must have at least 2 unique items for enumeration type.")
        return value

    @model_validator(mode='before')
    def validate_enumeration_values(cls, values):
        """
        Ensure enumeration_values is valid for enumeration type attributes.

        Raises:
            ValueError: If enumeration_values has fewer than 2 unique items.

        Returns:
            values: The validated values.
        """
        logger.debug(f"Validating attribute values: {values}")
        type_ = values.get('type')
        enumeration_values = values.get('enumeration_values')
        if type_ == 'enumeration':
            if not enumeration_values or len(enumeration_values) < 2:
                raise ValueError("enumeration_values must have at least 2 unique items for enumeration type.")
        return values

# Bind Model
class Bind(BaseModel):
    """
    Represents the binding of attributes to an activity.

    Attributes:
        activity (Activity): The activity involved in the binding.
        attributes (List[Attribute]): List of attribute objects bound to the activity.
        description (str): Description of the purpose or role of the bind.
    """
    activity: Activity = Field(
        ...,
        description="The activity which is present in the activity list."
    )
    attributes: conlist(Attribute, min_length=1) = Field(
        ...,
        description="List of attribute objects bound to the activity. Must have between 1 and a maximum of the total number of attributes. The attribute must have the min_value and max_value if it is integer or float and if it is enumeration it must have more than one option."
    )
    description: str = Field(
        ...,
        description="Description of the purpose or role of the bind."
    )

    @model_validator(mode='before')
    def validate_attributes(cls, values):
        """
        Ensure attributes are valid instances of Attribute.

        Raises:
            ValueError: If attributes are not valid instances of Attribute.

        Returns:
            values: The validated values.
        """
        logger.debug(f"Validating attributes for bind: {values}")
        attributes = values.get('attributes', [])
        for attribute in attributes:
            if not isinstance(attribute, Attribute):
                Attribute(**attribute)
        return values

# Constraint Model
class Constraint(BaseModel):
    """
    Represents a constraint in the process model.

    Attributes:
        type (str): The type of constraint. It can be unary or binary.
        description (str): A detailed description of what the constraint entails.
        template (str): The template of the constraint.
        activation (Optional[Activity]): The activation activity involved in the constraint.
        target (Optional[Activity]): The target activity involved in the constraint.
        activation_condition (Optional[str]): Condition to activate the constraint.
        correlation_condition (Optional[str]): Condition to correlate the constraint.
        time_condition (Optional[str]): Condition to specify time constraints.
        cardinality (Optional[conint(ge=1)]): The cardinality for the Existence, Absence, and Exactly templates.
        all_binds (Dict[str, Bind]): Dictionary of binds associated with the constraint.
    """
    type: str = Field(
        ...,
        description="The type of constraint. It can be unary or binary",
        pattern=r"^(unary|binary)$"
    )
    description: str = Field(
        ...,
        description="A detailed description of what the constraint entails."
    )
    template: str = Field(
        ...,
        description="The template of the constraint.",
        pattern=r"^(Init|End|Existence|Absence|Exactly|Alternate Precedence|Alternate Response|Alternate Succession|Chain Precedence|Chain Response|Chain Succession|Co-Existence|Precedence|Responded Existence|Response|Succession|Not Chain Succession|Not Co-Existence|Not Succession|Not Chain Precedence|Not Chain Response|Not Precedence|Not Responded Existence|Not Response|Choice|Exclusive Choice)$"
    )
    activation: Optional[Activity] = Field(
        None,
        description="The activation activity involved in the constraint."
    )
    target: Optional[Activity] = Field(
        None,
        description="The target activity involved in the constraint."
    )
    activation_condition: Optional[str] = Field(
        None,
        description="Condition to activate the constraint. It must reference attributes of the activation activity and use valid operations.",
        pattern=r"^A\.[a-zA-Z0-9_ ]+\s*(==|!=|>|>=|<|<=| in | not in | is | is not | and | or |\))"
    )
    correlation_condition: Optional[str] = Field(
        None,
        description="Condition to correlate the constraint. It must reference attributes of both activation and target activities and use valid operations.",
        pattern=r"^(A\.[a-zA-Z0-9_ ]+\s*(==|!=|>|>=|<|<=)\s*(T\.[a-zA-Z0-9_ ]+|\d+(\.\d+)?|'[^']*')|T\.[a-zA-Z0-9_ ]+\s*(==|!=|>|>=|<|<=)\s*(A\.[a-zA-Z0-9_ ]+|\d+(\.\d+)?|'[^']*'))(\s+(and|or)\s+(A\.[a-zA-Z0-9_ ]+\s*(==|!=|>|>=|<|<=)\s*(T\.[a-zA-Z0-9_ ]+|\d+(\.\d+)?|'[^']*')|T\.[a-zA-Z0-9_ ]+\s*(==|!=|>|>=|<|<=)\s*(A\.[a-zA-Z0-9_ ]+|\d+(\.\d+)?|'[^']*')))*$"
    )
    time_condition: Optional[str] = Field(
        None,
        description="Condition to specify time constraints.",
        pattern=r"^\d+,\d+,[smhd]$"
    )
    cardinality: Optional[conint(ge=1)] = Field(
        None,
        description="The cardinality for the Existence, Absence, and Exactly templates."
    )
    all_binds: Dict[str, Bind] = Field(
        default_factory=dict,
        exclude=True
    )

    @field_validator('activation', 'target', mode='before')
    def check_activation_and_target(cls, value, info):
        """
        Validate activation and target based on the type of constraint.

        Raises:
            ValueError: If activation or target is required but not provided.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Validating {info.field_name} for constraint type {info.data['type']}.")
        if info.data['type'] == 'binary':
            if info.field_name == 'activation' and value is None:
                raise ValueError("activation is required for binary type constraints.")
            if info.field_name == 'target' and value is None:
                raise ValueError("target is required for binary type constraints.")
        if info.data['type'] == 'unary' and info.field_name == 'activation' and value is None:
            raise ValueError("activation is required for unary type constraints.")
        return value

    @field_validator('cardinality', mode='before')
    def check_cardinality(cls, value, info):
        """
        Validate cardinality for certain templates.

        Raises:
            ValueError: If cardinality is required but not provided.

        Returns:
            value: The validated value.
        """
        logger.debug(f"Validating cardinality for template {info.data['template']}.")
        if info.data['template'] in ['Existence', 'Absence', 'Exactly'] and value is None:
            raise ValueError("cardinality is required for Existence, Absence, and Exactly templates.")
        return value

    @model_validator(mode='before')
    def validate_conditions(cls, values):
        """
        Validate conditions for constraints.

        Raises:
            ValueError: If conditions reference attributes not bound to the appropriate activities.

        Returns:
            values: The validated values.
        """
        logger.debug(f"Validating conditions for constraint with type {values.get('type')} and template {values.get('template')}")
        activation_condition = values.get('activation_condition')
        correlation_condition = values.get('correlation_condition')
        activation = values.get('activation')
        target = values.get('target')
        all_binds = values.get('all_binds', {})

        def get_condition_attributes(condition: str, prefix: str) -> set:
            if not condition:
                return set()
            pattern = re.compile(rf'\b{prefix}\.([a-zA-Z0-9_ ]+)(?=\s*(==|!=|>|>=|<|<=| in | not in | is | is not | and | or |\)|$))', re.IGNORECASE)
            attributes = set(m.group(1).strip() for m in re.finditer(pattern, condition))
            logger.debug(f"Extracted attributes from condition '{condition}' with prefix '{prefix}': {attributes}")
            return attributes

        def get_bound_attribute_names(activity_name: str) -> set:
            bind = all_binds.get(activity_name)
            if not bind:
                raise ValueError(f"No bind found for activity_name: {activity_name}")
            bound_attributes = set(attr.name for attr in bind.attributes)
            logger.debug(f"Bound attributes for activity '{activity_name}': {bound_attributes}")
            return bound_attributes

        if activation_condition and activation:
            logger.debug(f"Validating activation condition: {activation_condition} for activation activity: {activation.name}")
            extracted_activation_attrs = get_condition_attributes(activation_condition, 'A')
            bound_activation_attrs = get_bound_attribute_names(activation.name)
            invalid_activation_attrs = extracted_activation_attrs - bound_activation_attrs
            if invalid_activation_attrs:
                raise ValueError(f"Attributes in activation_condition {invalid_activation_attrs} are not bound to the activation activity {activation.name}")

        if correlation_condition and activation and target:
            logger.debug(f"Validating correlation condition: {correlation_condition} for activation activity: {activation.name} and target activity: {target.name}")
            extracted_activation_attrs = get_condition_attributes(correlation_condition, 'A')
            extracted_target_attrs = get_condition_attributes(correlation_condition, 'T')
            bound_activation_attrs = get_bound_attribute_names(activation.name)
            bound_target_attrs = get_bound_attribute_names(target.name)
            invalid_correlation_attrs = (extracted_activation_attrs - bound_activation_attrs) | (extracted_target_attrs - bound_target_attrs)
            if invalid_correlation_attrs:
                raise ValueError(f"Attributes in correlation_condition {invalid_correlation_attrs} are not bound to the appropriate activities")

        return values

# MPDeclareModel
class MPDeclareModel(BaseModel):
    """
    Represents a process model with activities, attributes, binds, and constraints.

    Attributes:
        activities (List[Activity]): List of activities in the process model.
        attributes (List[Attribute]): Attributes used in the process model.
        binds (List[Bind]): Links between activities and their attributes.
        constraints (List[Constraint]): Constraints that rule the process execution.
    """
    activities: List[Activity] = Field(
        ...,
        description="List of activities in the process model.",
        examples=[
            [
                {"name": "Log Ticket", "description": "Record a new support ticket."},
                {"name": "Assign Ticket", "description": "Assign the ticket to a support agent."},
                {"name": "Resolve Ticket", "description": "Resolve the customer's issue."},
                {"name": "Verify Resolution", "description": "Verify that the issue has been resolved."},
                {"name": "Close Ticket", "description": "Close the support ticket."},
                {"name": "Send Survey", "description": "Send a survey to the customer."},
                {"name": "Archive Ticket", "description": "Archive the ticket and related data."}
            ]
        ]
    )
    attributes: List[Attribute] = Field(
        ...,
        description="Attributes used in the process model.",
        examples=[
            [
                {"type": "integer", "name": "Ticket ID", "description": "Unique identifier for the support ticket.", "min_value": 1, "max_value": 999999},
                {"type": "enumeration", "name": "Customer Name", "description": "Name of the customer.", "enumeration_values": ["John Doe", "Jane Smith", "Alice Johnson"]},
                {"type": "enumeration", "name": "Issue Description", "description": "Description of the issue reported by the customer.", "enumeration_values": ["Cannot login", "Payment issue", "Technical problem"]},
                {"type": "integer", "name": "Agent ID", "description": "Unique identifier for the support agent.", "min_value": 1, "max_value": 9999},
                {"type": "enumeration", "name": "Assignment Date", "description": "Date when the ticket was assigned.", "enumeration_values": ["2023-06-01", "2023-06-02", "2023-06-03"]},
                {"type": "enumeration", "name": "Resolution Details", "description": "Details of the resolution provided.", "enumeration_values": ["Password reset", "Refund processed", "Technical fix applied"]},
                {"type": "enumeration", "name": "Resolution Date", "description": "Date when the issue was resolved.", "enumeration_values": ["2023-06-04", "2023-06-05", "2023-06-06"]},
                {"type": "enumeration", "name": "Verification Status", "description": "Status of the resolution verification.", "enumeration_values": ["Verified", "Not Verified"]},
                {"type": "integer", "name": "Verifier ID", "description": "Unique identifier for the verifier.", "min_value": 1, "max_value": 9999},
                {"type": "enumeration", "name": "Close Date", "description": "Date when the ticket was closed.", "enumeration_values": ["2023-06-07", "2023-06-08", "2023-06-09"]},
                {"type": "integer", "name": "Survey ID", "description": "Unique identifier for the survey.", "min_value": 1, "max_value": 999999},
                {"type": "enumeration", "name": "Survey Date", "description": "Date when the survey was sent.", "enumeration_values": ["2023-06-10", "2023-06-11", "2023-06-12"]},
                {"type": "enumeration", "name": "Archive Date", "description": "Date when the ticket was archived.", "enumeration_values": ["2023-06-13", "2023-06-14", "2023-06-15"]}
            ]
        ]
    )
    binds: List[Bind] = Field(
        ...,
        description="Links between activities and their attributes.",
        examples=[
            [
                {"activity": {"name": "Log Ticket", "description": "Record a new support ticket."}, "attributes": [{"type": "integer", "name": "Ticket ID", "description": "Unique identifier for the support ticket.", "min_value": 1, "max_value": 999999}, {"type": "enumeration", "name": "Customer Name", "description": "Name of the customer.", "enumeration_values": ["John Doe", "Jane Smith", "Alice Johnson"]}, {"type": "enumeration", "name": "Issue Description", "description": "Description of the issue reported by the customer.", "enumeration_values": ["Cannot login", "Payment issue", "Technical problem"]}], "description": "Bind Ticket ID, Customer Name, and Issue Description to 'Log Ticket' activity."},
                {"activity": {"name": "Assign Ticket", "description": "Assign the ticket to a support agent."}, "attributes": [{"type": "integer", "name": "Ticket ID", "description": "Unique identifier for the support ticket.", "min_value": 1, "max_value": 999999}, {"type": "integer", "name": "Agent ID", "description": "Unique identifier for the support agent.", "min_value": 1, "max_value": 9999}, {"type": "enumeration", "name": "Assignment Date", "description": "Date when the ticket was assigned.", "enumeration_values": ["2023-06-01", "2023-06-02", "2023-06-03"]}], "description": "Bind Ticket ID, Agent ID, and Assignment Date to 'Assign Ticket' activity."},
                {"activity": {"name": "Resolve Ticket", "description": "Resolve the customer's issue."}, "attributes": [{"type": "integer", "name": "Ticket ID", "description": "Unique identifier for the support ticket.", "min_value": 1, "max_value": 999999}, {"type": "enumeration", "name": "Resolution Details", "description": "Details of the resolution provided.", "enumeration_values": ["Password reset", "Refund processed", "Technical fix applied"]}, {"type": "enumeration", "name": "Resolution Date", "description": "Date when the issue was resolved.", "enumeration_values": ["2023-06-04", "2023-06-05", "2023-06-06"]}], "description": "Bind Ticket ID, Resolution Details, and Resolution Date to 'Resolve Ticket' activity."},
                {"activity": {"name": "Verify Resolution", "description": "Verify that the issue has been resolved."}, "attributes": [{"type": "integer", "name": "Ticket ID", "description": "Unique identifier for the support ticket.", "min_value": 1, "max_value": 999999}, {"type": "enumeration", "name": "Verification Status", "description": "Status of the resolution verification.", "enumeration_values": ["Verified", "Not Verified"]}, {"type": "integer", "name": "Verifier ID", "description": "Unique identifier for the verifier.", "min_value": 1, "max_value": 9999}], "description": "Bind Ticket ID, Verification Status, and Verifier ID to 'Verify Resolution' activity."},
                {"activity": {"name": "Close Ticket", "description": "Close the support ticket."}, "attributes": [{"type": "integer", "name": "Ticket ID", "description": "Unique identifier for the support ticket.", "min_value": 1, "max_value": 999999}, {"type": "enumeration", "name": "Close Date", "description": "Date when the ticket was closed.", "enumeration_values": ["2023-06-07", "2023-06-08", "2023-06-09"]}], "description": "Bind Ticket ID and Close Date to 'Close Ticket' activity."},
                {"activity": {"name": "Send Survey", "description": "Send a survey to the customer."}, "attributes": [{"type": "integer", "name": "Survey ID", "description": "Unique identifier for the survey.", "min_value": 1, "max_value": 999999}, {"type": "integer", "name": "Ticket ID", "description": "Unique identifier for the support ticket.", "min_value": 1, "max_value": 999999}, {"type": "enumeration", "name": "Survey Date", "description": "Date when the survey was sent.", "enumeration_values": ["2023-06-10", "2023-06-11", "2023-06-12"]}], "description": "Bind Survey ID, Ticket ID, and Survey Date to 'Send Survey' activity."},
                {"activity": {"name": "Archive Ticket", "description": "Archive the ticket and related data."}, "attributes": [{"type": "integer", "name": "Ticket ID", "description": "Unique identifier for the support ticket.", "min_value": 1, "max_value": 999999}, {"type": "enumeration", "name": "Archive Date", "description": "Date when the ticket was archived.", "enumeration_values": ["2023-06-13", "2023-06-14", "2023-06-15"]}], "description": "Bind Ticket ID and Archive Date to 'Archive Ticket' activity."}
            ]
        ]
    )
    constraints: List[Constraint] = Field(
        ...,
        description="Constraints that rule the process execution.",
        examples=[
            [
                {"type": "unary", "description": "Ticket logging must start the process", "template": "Init", "activation": {"name": "Log Ticket", "description": "Record a new support ticket."}},
                {"type": "binary", "description": "Ticket assignment must occur after logging", "template": "Precedence", "activation": {"name": "Log Ticket", "description": "Record a new support ticket."}, "target": {"name": "Assign Ticket", "description": "Assign the ticket to a support agent."}},
                {"type": "binary", "description": "Ticket resolution must occur after assignment", "template": "Response", "activation": {"name": "Assign Ticket", "description": "Assign the ticket to a support agent."}, "target": {"name": "Resolve Ticket", "description": "Resolve the customer's issue."}},
                {"type": "binary", "description": "Resolution verification must immediately follow resolution", "template": "Chain Succession", "activation": {"name": "Resolve Ticket", "description": "Resolve the customer's issue."}, "target": {"name": "Verify Resolution", "description": "Verify that the issue has been resolved."}},
                {"type": "binary", "description": "Ticket must be closed within 24 hours after verification", "template": "Precedence", "activation": {"name": "Verify Resolution", "description": "Verify that the issue has been resolved."}, "target": {"name": "Close Ticket", "description": "Close the support ticket."}, "time_condition": "0,24,h"},
                {"type": "binary", "description": "Survey must be sent within 48 hours after ticket closure", "template": "Alternate Response", "activation": {"name": "Close Ticket", "description": "Close the support ticket."}, "target": {"name": "Send Survey", "description": "Send a survey to the customer."}, "time_condition": "0,48,h"},
                {"type": "unary", "description": "Process must end after ticket is archived", "template": "End", "activation": {"name": "Archive Ticket", "description": "Archive the ticket and related data."}}
            ]
        ]
    )

    @model_validator(mode='before')
    def attach_binds_to_constraints(cls, values):
        """
        Attach binds to constraints in the process model.

        Raises:
            ValueError: If binds for activation and target activities are not found.

        Returns:
            values: The validated values with attached binds.
        """
        logger.debug("Attaching binds to constraints in ProcessModel.")
        bind_map = {}
        for bind in values.get('binds', []):
            logger.debug(f"Processing bind for activity: {bind['activity']['name']}")
            activity = Activity(**bind['activity'])
            attributes = [Attribute(**attr) for attr in bind['attributes']]
            bind_instance = Bind(activity=activity, attributes=attributes, description=bind['description'])
            bind_map[activity.name] = bind_instance
        constraints = values.get('constraints', [])
        new_constraints = []
        for constraint_data in constraints:
            if 'activation' in constraint_data and isinstance(constraint_data['activation'], dict):
                logger.debug(f"Converting activation from dict to Activity for constraint with description: {constraint_data['description']}")
                constraint_data['activation'] = Activity(**constraint_data['activation'])
            if 'target' in constraint_data and isinstance(constraint_data['target'], dict):
                logger.debug(f"Converting target from dict to Activity for constraint with description: {constraint_data['description']}")
                constraint_data['target'] = Activity(**constraint_data['target'])

            # Filter binds for activation and target activities only
            filtered_binds = {k: v for k, v in bind_map.items() if k in [constraint_data['activation'].name, constraint_data.get('target').name if constraint_data.get('target') else None]}
            constraint_data['all_binds'] = filtered_binds

            constraint = Constraint(**constraint_data)
            new_constraints.append(constraint)
        values['constraints'] = new_constraints
        return values

    def convert_to_string(self) -> str:
        """
        Convert the process model to a string representation.

        Returns:
            str: The string representation of the process model.
        """
        # Convert activities
        activities_str = "\n".join(
            [f"activity {activity.name}" for activity in self.activities]
        )

        # Convert binds
        binds_str = "\n".join(
            [
                f"bind {bind.activity.name}: " + ", ".join([attr.name for attr in bind.attributes])
                for bind in self.binds
            ]
        )

        # Convert attributes
        attributes_str = ""
        for attribute in self.attributes:
            if attribute.type in ['integer', 'float']:
                attributes_str += f"{attribute.name}: {attribute.type} between {attribute.min_value} and {attribute.max_value}\n"
            elif attribute.type == 'enumeration':
                enumeration_values = ", ".join(attribute.enumeration_values)
                attributes_str += f"{attribute.name}: {enumeration_values}\n"

        # Convert constraints
        constraints_str = ""
        for constraint in self.constraints:
            if constraint.type == 'unary':
                constraint_str = f"{constraint.template}[{constraint.activation.name}]"
                if constraint.cardinality and constraint.cardinality > 1:
                    constraint_str += f"{constraint.cardinality}"
                constraint_str += f" |{constraint.activation_condition or ''} |{constraint.time_condition or ''}"
            elif constraint.type == 'binary':
                constraint_str = f"{constraint.template}[{constraint.activation.name}, {constraint.target.name}]"
                constraint_str += f" |{constraint.activation_condition or ''} |{constraint.correlation_condition or ''} |{constraint.time_condition or ''}"
            constraints_str += f"{constraint_str}\n"

        # Combine all parts
        final_str = "\n".join([activities_str, binds_str, attributes_str.strip(), constraints_str.strip()])

        return final_str


In [ ]:
# Inspecting the json schema
print(MPDeclareModel.model_json_schema())

In [ ]:
process="""
## Overall Activity Context

The business process described here is centered around organizing and executing an event. The process encompasses a wide range of activities, from initial planning to post-event feedback collection. Each activity is intricately linked to specific attributes and is governed by a series of constraints that ensure a logical and efficient flow. The primary goal is to ensure that the event is meticulously planned, smoothly executed, and thoroughly reviewed.

## Process Flow Description

### Initiation

The process begins with the **Plan Event** activity, which is the initial phase of organizing an event. This activity is bound to the attributes **Event ID** and **Budget**, ensuring that the planning is associated with a specific event and its allocated budget. The constraint **Init[Plan Event]** mandates that this activity must start the process.

### Venue Booking

Following the planning phase, the **Book Venue** activity is executed. This activity is linked to the attributes **Event ID** and **Venue**, ensuring that the booking process is tied to a specific event and location. The constraint **Precedence[Plan Event, Book Venue]** ensures that venue booking occurs only after the event planning phase is completed.

### Sending Invitations

Once the venue is secured, the **Send Invitations** activity takes place. This activity is associated with the attributes **Event ID**, **Number of Guests**, and **Invitation Status**, indicating that the invitations are sent to a specific number of guests for the event. The constraint **Precedence[Book Venue, Send Invitations]** ensures that invitations are sent only after the venue is booked.

### Arranging Catering

After invitations are sent, the **Arrange Catering** activity is executed. This activity is linked to the attributes **Event ID** and **Catering Service**, ensuring that catering arrangements are made for a specific event and type of service. The constraint **Precedence[Send Invitations, Arrange Catering]** ensures that catering arrangements are made only after invitations are sent.

### Hiring Staff

Following the catering arrangements, the **Hire Staff** activity takes place. This activity is associated with the attributes **Event ID** and **Staff Role**, indicating that staff members are hired for specific roles at the event. The constraint **Precedence[Arrange Catering, Hire Staff]** ensures that staff hiring occurs only after catering arrangements are made.

### Setting Up the Venue

Once the staff is hired, the **Set Up Venue** activity is executed. This activity involves preparing the venue for the event, including decorations and equipment setup. The constraint **Precedence[Hire Staff, Set Up Venue]** ensures that venue setup occurs only after staff hiring is completed.

### Conducting Rehearsal

After the venue is set up, the **Conduct Rehearsal** activity takes place. This activity is linked to the attributes **Event ID** and **Rehearsal ID**, ensuring that the rehearsal is tied to a specific event and its rehearsal identifier. The constraint **Precedence[Set Up Venue, Conduct Rehearsal]** ensures that rehearsals are conducted only after the venue is set up.

### Welcoming Guests

Following the rehearsal, the **Welcome Guests** activity is executed. This activity involves greeting and registering guests as they arrive at the event. The constraint **Precedence[Conduct Rehearsal, Welcome Guests]** ensures that guests are welcomed only after rehearsals are conducted.

### Facilitating Activities

Once guests are welcomed, the **Facilitate Activities** activity takes place. This activity includes managing and overseeing the planned activities during the event. The constraint **Precedence[Welcome Guests, Facilitate Activities]** ensures that activities are facilitated only after guests are welcomed.

### Delivering Speech

Following the facilitation of activities, the **Deliver Speech** activity is executed. This activity involves giving a speech or presentation during the event. The constraint **Precedence[Facilitate Activities, Deliver Speech]** ensures that speeches are delivered only after activities are facilitated.

### Serving Meals

After the speech, the **Serve Meals** activity takes place. This activity is linked to the attributes **Event ID** and **Catering Service**, ensuring that meals are served for a specific event and type of service. The constraint **Precedence[Deliver Speech, Serve Meals]** ensures that meals are served only after speeches are delivered.

### Coordinating Photography

Following the meal service, the **Coordinate Photography** activity is executed. This activity involves managing photography services to capture moments during the event. The constraint **Precedence[Serve Meals, Coordinate Photography]** ensures that photography is coordinated only after meals are served.

### Distributing Gifts

Once photography is coordinated, the **Distribute Gifts** activity takes place. This activity includes handing out gifts or souvenirs to guests. The constraint **Precedence[Coordinate Photography, Distribute Gifts]** ensures that gifts are distributed only after photography is coordinated.

### Collecting Feedback

After gifts are distributed, the **Collect Feedback** activity is executed. This activity is linked to the attributes **Event ID** and **Venue**, ensuring that feedback is collected for a specific event and location. The constraint **Precedence[Distribute Gifts, Collect Feedback]** ensures that feedback is collected only after gifts are distributed.

### Tearing Down the Venue

The process concludes with the **Tear Down Venue** activity, which involves dismantling decorations and equipment after the event. The constraint **End[Tear Down Venue]** ensures that this activity marks the end of the event process.

## Constraint Integration

### Unary Constraints

- **Init[Plan Event]**: Ensures that the 'Plan Event' activity initiates the process.
- **End[Tear Down Venue]**: Ensures that the 'Tear Down Venue' activity concludes the process.

### Binary Constraints

- **Precedence[Plan Event, Book Venue]**: Ensures that venue booking follows event planning.
- **Precedence[Book Venue, Send Invitations]**: Ensures that invitations are sent after the venue is booked.
- **Precedence[Send Invitations, Arrange Catering]**: Ensures that catering arrangements follow the sending of invitations.
- **Precedence[Arrange Catering, Hire Staff]**: Ensures that staff hiring follows catering arrangements.
- **Precedence[Hire Staff, Set Up Venue]**: Ensures that venue setup follows staff hiring.
- **Precedence[Set Up Venue, Conduct Rehearsal]**: Ensures that rehearsals follow venue setup.
- **Precedence[Conduct Rehearsal, Welcome Guests]**: Ensures that guests are welcomed after rehearsals.
- **Precedence[Welcome Guests, Facilitate Activities]**: Ensures that activities are facilitated after guests are welcomed.
- **Precedence[Facilitate Activities, Deliver Speech]**: Ensures that speeches are delivered after activities are facilitated.
- **Precedence[Deliver Speech, Serve Meals]**: Ensures that meals are served after speeches are delivered.
- **Precedence[Serve Meals, Coordinate Photography]**: Ensures that photography is coordinated after meals are served.
- **Precedence[Coordinate Photography, Distribute Gifts]**: Ensures that gifts are distributed after photography is coordinated.
- **Precedence[Distribute Gifts, Collect Feedback]**: Ensures that feedback is collected after gifts are distributed.

## Conclusion

This business process model meticulously outlines the sequence of activities required to organize and execute an event, from initial planning to post-event feedback collection. The constraints ensure a logical and efficient flow, with each activity dependent on the successful completion of preceding activities. The integration of attributes and binds further personalizes each activity to the specific event, ensuring that all aspects of the event are carefully managed and executed. The most critical constraints, such as the initiation and conclusion activities, along with the precedence constraints, ensure that the process is both structured and flexible, allowing for a seamless event execution."""

##Declare4py Object Instantiation

It's important to note that the MPDeclareModel object are not compliant to declare4py objects. Hence, we need to instantiate it. In the foundational data model we created a function that converts the instance of it to a string. Here, we extend the class DeclareModel from declare4py to add two new methods that were not previously implemented: get attributes and get binds. These methods will be useful to interface with prompts.

In [ ]:
#Example model to generate descriptions
model ="""activity Plan Event
activity Book Venue
activity Send Invitations
activity Arrange Catering
activity Hire Staff
activity Set Up Venue
activity Conduct Rehearsal
activity Welcome Guests
activity Facilitate Activities
activity Deliver Speech
activity Serve Meals
activity Coordinate Photography
activity Distribute Gifts
activity Tear Down Venue
activity Collect Feedback
bind Plan Event: Event ID, Budget
bind Book Venue: Event ID, Venue
bind Send Invitations: Event ID, Number of Guests, Invitation Status
bind Arrange Catering: Event ID, Catering Service
bind Hire Staff: Event ID, Staff Role
bind Conduct Rehearsal: Event ID, Rehearsal ID
bind Serve Meals: Event ID, Catering Service
bind Collect Feedback: Event ID, Venue
Event ID: integer between 1 and 999999
Venue: Hall A, Hall B, Hall C
Catering Service: Buffet, Sit-down Dinner, Cocktail
Number of Guests: integer between 10 and 500
Invitation Status: Sent, Not Sent
Staff Role: Waiter, Coordinator, Cleaner
Rehearsal ID: integer between 1 and 9999
Budget: float between 1000 and 50000
Init[Plan Event] | |
Precedence[Plan Event, Book Venue] | | |
Precedence[Book Venue, Send Invitations] | | |
Precedence[Send Invitations, Arrange Catering] | | |
Precedence[Arrange Catering, Hire Staff] | | |
Precedence[Hire Staff, Set Up Venue] | | |
Precedence[Set Up Venue, Conduct Rehearsal] | | |
Precedence[Conduct Rehearsal, Welcome Guests] | | |
Precedence[Welcome Guests, Facilitate Activities] | | |
Precedence[Facilitate Activities, Deliver Speech] | | |
Precedence[Deliver Speech, Serve Meals] | | |
Precedence[Serve Meals, Coordinate Photography] | | |
Precedence[Coordinate Photography, Distribute Gifts] | | |
Precedence[Distribute Gifts, Collect Feedback] | | |
End[Tear Down Venue] | |
"""

In [ ]:
#Instanciating the ExtendedDeclareModel with the string of the MPDeclareModel.
declare_model = ExtendedDeclareModel().parse_from_string(model)

# Fetching activities, constraints, attributes, and binds from the model
model_activities = declare_model.get_model_activities()
model_attributes = declare_model.get_decl_attributes()
model_binds = declare_model.get_decl_binds()
model_constraints = declare_model.get_decl_model_constraints()
print("MP-Declare Model Constructs")
# Printing model activities
print("Model activities:")
print("-----------------")
for idx, act in enumerate(model_activities):
    print(idx, act)
print("\n")

# Printing model attributes
print("Model attributes:")
print("-----------------")
for idx, attr in enumerate(model_attributes):
    print(idx, attr)
print("\n")

# Printing model binds
print("Model binds:")
print("-------------")
for idx, bind in enumerate(model_binds):
    print(idx, bind)
print("\n")

# Printing model constraints
print("Model constraints:")
print("-----------------")
for idx, constr in enumerate(model_constraints):
    print(idx, constr)
print("\n")


It is now compliant with our needs.

##MP-Declare to Intermediary Description Generation

The goal of this step is to generate intermediary descriptions of MP-Declare constructs. This is essential to have semantically enriched descriptions which can be used to evaluate if each part of the model is covered semantically and with correctness and completeness.

To achieve a consistent format of the intermediary descriptions we used some prompt engineering techniques: role-play prompting, knowledge infusion, few-shot learning and instruction tunning. Prompt engineering leverages large language models' capabilities without no need to fine-tune the model to achieve a goal.

In [ ]:
# The main prompt used to generate the intermediary descriptions.
decltotextassistant="""
You are an AI configured as an expert in declarative business process management, specifically for interpreting and describing Multi-Perspective Declare models (MP-Declare). Your primary task is to analyze the provided MP-Declare model and generate a comprehensive and consistent description. This description should meticulously reflect the structure and terminology of the model, maintaining exact naming and case sensitivity for model parts (activities, attributes, binds, and constraints). Examples of model descriptions are provided below (name of construct and example separated by ;) to guide your output and ensure alignment between data representation and text description. Your output will be the name of the construct and generated description.

# Example 1

- MP-Declare Model:

activity login
activity add_to_basket
activity remove_from_basket
activity confirmation
activity pay
activity get_discount
activity payment_success
activity order
activity shipping

bind login: user_id, password, confirm, user_type
bind add_to_basket: product, quantity, price
bind remove_from_basket: product, quantity
bind confirmation: confirm
bind pay: price, success, quantity, product
bind payment_success: confirm
bind order: status
bind shipping: shipped
bind get_discount: discount

user_id: integer between 1 and 100
password: float between 1000000 and 9000000
discount: integer between 5 and 15
quantity: integer between 1 and 50
product: cloths, radio, computer, notebook, food, chocolate
shipped: true, false
success: true, false
confirm: true, false
price: float between 0.99 and 5000.0
status: approved, cancelled, pending
user_type: admin, customer, delivery_boy


existence[login] | A.confirm is true |
Precedence[pay, confirmation] | | |

Response[pay, get_discount] |A.success is true and (A.price > 1000 or (A.product in (computer, notebook))) | T.discount = 10|
Response[pay, get_discount] |A.success is true and (A.price > 50 or (A.product in (chocolate, food))) | T.discount = 2|
Response[pay, get_discount] |A.price > 100 and (A.price < 1000) and A.success is true | T.discount = 5|
Response[pay, get_discount] | (A.success is false or A.quantity <= 0) and A.product not in (computer, notebook) or ((A.product is not food) and (A.product is not cloths)) | T.discount = 0|
Response[pay, get_discount] |A.price > 1500 and A.success is true | T.discount = 5|
Response[pay, order] |A.success is true | T.status is approved|
Response[pay, order] |A.success is false | T.status is cancelled|
Response[order, shipping] |A.status is approved | T.shipped is true|
Response[payment_success, get_discount] | | T.discount > 1|
Response[add_to_basket, pay] |  | |
Precedence[pay, login] | |T.user_type is customer|
Precedence[payment_success, pay] | |T.price >= 0.99|
Precedence[remove_from_basket, add_to_basket] | | |

- Description:

Activities:
- Name: login; Description: The 'login' activity represents the initial user authentication.
- Name: add_to_basket; Description: The 'add_to_basket' activity represents adding items to the shopping basket.
- Name: remove_from_basket; Description: The 'remove_from_basket' activity represents removing items from the shopping basket.
- Name: confirmation; Description: The 'confirmation' activity represents confirming order or payment details.
- Name: pay; Description: The 'pay' activity represents processing payment for items.
- Name: get_discount; Description: The 'get_discount' activity represents applying discounts based on specific conditions.
- Name: payment_success; Description: The 'payment_success' activity represents successful payment processing.
- Name: order; Description: The 'order' activity represents managing the status of an order.
- Name: shipping; Description: The 'shipping' activity represents dispatching the order to the customer.


Attributes:
- Name: user_id: Integer between 1 and 100; Description: The attribute 'user_id: integer between 1 and 100' represents the user's identification number for authentication purposes.
- Name: password: Float between 1000000 and 9000000; Description: The attribute 'password: float between 1000000 and 9000000' represents the user's password for login.
- Name: product: Cloths, radio, computer, notebook, food, chocolate; Description: The attribute 'product: cloths, radio, computer, notebook, food, chocolate' specifies the types of products available for purchase.
- Name: price: Float between 0.99 and 5000.0; Description: The attribute 'price: float between 0.99 and 5000.0' indicates the price range of the products, from 0.99 to 5000 dollars.
- Name: discount: Integer between 5 and 15; Description: The attribute 'discount: integer between 5 and 15' denotes the range of discount that can be applied to a transaction.
- Name: quantity: Integer between 1 and 50; Description: The attribute 'quantity: integer between 1 and 50' represents the number of units of a product a user is purchasing.
- Name: shipped: True, False; Description: The attribute 'shipped: true, false' indicates whether the goods have been shipped.
- Name: success: True, False; Description: The attribute 'success: true, false' indicates the outcome of a transaction which can be True or False.
- Name: confirm: True, False; Description: The attribute 'confirm: true, false' indicates whether the transaction or delivery has been confirmed by the user and can be True or False.
- Name: user_type: Admin, customer, delivery_boy; Description: The attribute 'user_type: admin, customer, delivery_boy' describes the role of the user within the system and can be admin, customer or delivery_boy.
- Name: status: approved, cancelled, pending; Description: The attribute 'status: approved, cancelled, pending' indicates wheter the order is status with possibles values being approved, cancelled or pendind.


Binds:
- Name: login: user_id, password, confirm, user_type; Description: This bind connects the 'login' activity with the attributes 'user_id', 'password', 'confirm', and 'user_type', ensuring that user authentication is tied to these specific credentials and roles.
- Name: add_to_basket: product, quantity, price; Description: This bind links the 'add_to_basket' activity with the attributes 'product', 'quantity', and 'price', associating the addition of items to the basket with their specific types, quantities, and prices.
- Name: remove_from_basket: product, quantity; Description: This bind associates the 'remove_from_basket' activity with the attributes 'product' and 'quantity', linking the removal of items from the basket with the specific product and the quantity removed.
- Name: confirmation: confirm; Description: This bind ties the 'confirmation' activity to the 'confirm' attribute, indicating that confirmation is dependent on this boolean status.
- Name: pay: price, success, quantity, product; Description: This bind relates the 'pay' activity to the attributes 'price', 'success', 'quantity', and 'product', connecting payment processing to these specific details of the transaction.
- Name: payment_success: confirm; Description: This bind connects the 'payment_success' activity to the 'confirm' attribute, indicating that successful payment acknowledgment is tied to the confirmation status.
- Name: order: status; Description: This bind associates the 'order' activity with the 'status' attribute, linking order management to the current status of the order (approved, cancelled, pending).
- Name: shipping: shipped; Description: This bind links the 'shipping' activity to the 'shipped' attribute, indicating that the shipping process is dependent on whether the order has been dispatched.
- Name: get_discount: discount; Description: This bind couples the 'get_discount' activity to the 'discount' attribute, associating the application of discounts with the amount awarded.


Constraints:
- Name: existence[login] | A.confirm is true |; Description: This constraint ensures that the 'login' activity must exist if the attribute 'confirm' is true. It guarantees that a login is required whenever confirmation is provided.
- Name: Precedence[pay, confirmation] | | |; Description: This constraint specifies that the 'pay' activity can only occur after the 'confirmation' activity, ensuring that payment is processed only after order confirmation.
- Name: Response[pay, get_discount] | A.success is true and (A.price > 1000 or A.product in (computer, notebook)) | T.discount = 10 |; Description: This constraint applies a 10% discount after the 'pay' activity if the payment is successful and the 'price' is over $1000 or the 'product' is a computer or notebook, linking payment success and product type to the discount.
- Name: Response[pay, get_discount] | A.success is true and (A.price > 50 or A.product in (chocolate, food)) | T.discount = 2 |; Description: This constraint applies a 2% discount after the 'pay' activity if the payment is successful and the 'price' is over $50 or the 'product' is chocolate or food, linking payment success and product type to the discount.
- Name: Response[pay, get_discount] | A.price > 100 and A.price < 1000 and A.success is true | T.discount = 5 |; Description: This constraint grants a 5% discount after the 'pay' activity if the payment is successful and the 'price' is between $100 and $1000, establishing an intermediate price range for the discount.
- Name: Response[pay, get_discount] | (A.success is false or A.quantity <= 0) and A.product not in (computer, notebook) and A.product not in (food, cloths) | T.discount = 0 |; Description: This constraint ensures no discount is applied after the 'pay' activity if the payment fails, the 'quantity' is zero or negative, or the 'product' is not a computer, notebook, food, or clothes, preventing discounts in invalid situations.
- Name: Response[pay, get_discount] | A.price > 1500 and A.success is true | T.discount = 5 |; Description: This constraint awards a 5% discount after the 'pay' activity if the payment is successful and the 'price' is over $1500, promoting discounts for high-value transactions.
- Name: Response[pay, order] | A.success is true | T.status = approved |; Description: This constraint changes the 'order' status to 'approved' after the 'pay' activity if the payment is successful, linking payment success directly to order approval.
- Name: Response[pay, order] | A.success is false | T.status = cancelled |; Description: This constraint changes the 'order' status to 'cancelled' after the 'pay' activity if the payment fails, linking payment failure to order cancellation.
- Name: Response[order, shipping] | A.status = approved | T.shipped = true |; Description: This constraint ensures the 'shipping' activity occurs after the 'order' activity if the order status is 'approved', ensuring only approved orders are shipped.
- Name: Response[payment_success, get_discount] | | T.discount > 1 |; Description: This constraint applies a discount greater than 1% after the 'payment_success' activity, ensuring a discount is applied following successful payment.
- Name: Response[add_to_basket, pay] | | |; Description: This constraint requires the 'pay' activity to follow the 'add_to_basket' activity, establishing a logical sequence from adding items to the basket to making a payment.
- Name: Precedence[pay, login] | | T.user_type = customer |; Description: This constraint ensures the 'pay' activity follows the 'login' activity, confirming the user is a customer, and allowing only customers to make payments.
- Name: Precedence[payment_success, pay] | | T.price >= 0.99 |; Description: This constraint ensures the 'payment_success' activity follows the 'pay' activity, with a minimum 'price' of $0.99, setting a minimum transaction value for payment success.
- Name: Precedence[remove_from_basket, add_to_basket] | | |; Description: This constraint requires the 'remove_from_basket' activity to follow the 'add_to_basket' activity, establishing the correct order for managing the shopping basket.

# Example 2
- MP-Declare Model:
activity Apply for Trip
activity Approve application
activity Book means of transport
activity Book accommodation
activity Collect tickets
activity Archive documents


bind Apply for Trip: Application
bind Collect tickets: TicketCollection
bind Book means of transport: org::resource
bind Book accommodation: org::resource
bind Collect tickets: org::resource
bind Archive documents: org::resource
bind Apply for Trip: org::resource

Application: Application.written
TicketCollection: TicketCollection.written
org::resource: Alice, Bob


Precedence[Approve application, Apply for Trip B]||B.Application is Application.written
Precedence[Archive documents, Apply for Trip B]||B.Application is Application.written
Precedence[Archive documents, Collect tickets B]||B.TicketCollection is TicketCollection.written

Precedence[Book means of transport, Approve application]
Precedence[Book accommodation, Approve application]
Precedence[Collect tickets, Book means of transport]
Precedence[Collect tickets, Book accommodation]

Existence[Apply for Trip]
Existence[Approve application]
Existence[Collect tickets]
Existence[Archive documents]

- Description:

Activities:
- Name: Apply for Trip; Description: The 'Apply for Trip' activity represents the initial action where an individual submits an application for a trip.
- Name: Approve application; Description: The 'Approve application' activity involves the review and approval of the trip application.
- Name: Book means of transport; Description: The 'Book means of transport' activity entails making arrangements for transportation for the trip.
- Name: Book accommodation; Description: The 'Book accommodation' activity involves securing lodging for the duration of the trip.
- Name: Collect tickets; Description: The 'Collect tickets' activity represents the action of collecting tickets for transport and possibly for accommodations or events.
- Name: Archive documents; Description: The 'Archive documents' activity involves the process of organizing and storing documents related to the trip, such as receipts, tickets, and approvals, after the trip has concluded.

Attributes:
- Name: Application: Application.written; Description: The attribute 'Application: Application.written' indicates whether the trip application has been formally documented and submitted with the status 'written'.
- Name: TicketCollection: TicketCollection.written; Description: The attribute 'TicketCollection: TicketCollection.written' indicates the status 'written', focusing on whether the process of collecting tickets has been documented.
- Name: org::resource: Alice, Bob; Description: The attribute 'org::resource: Alice, Bob' represents the organizational resources involved in the trip planning process, which includes individuals named Alice and Bob.

Binds:
- Name: Apply for Trip: Application; Description: This bind links the 'Apply for Trip' activity with the attribute 'Application.written', indicating that the activity relies on the status of the application being formally documented and submitted.
- Name: Apply for Trip: org::resource; Description: This bind connects the 'Apply for Trip' activity with the organizational resources 'Alice, Bob', specifying that these individuals are involved in the application process.
- Name: Collect tickets: TicketCollection; Description: This bind associates the 'Collect tickets' activity with the attribute 'TicketCollection.written', highlighting that the collection process is documented.
- Name: Collect tickets: org::resource; Description: This bind links the 'Collect tickets' activity with the organizational resources 'Alice, Bob', indicating their involvement in the ticket collection process.
- Name: Book means of transport: org::resource; Description: This bind connects the 'Book means of transport' activity with the attribute 'org::resource' which represents organizational resources 'Alice, Bob', suggesting their responsibility in arranging transportation.
- Name: Book accommodation: org::resource; Description: This bind links the 'Book accommodation' activity with the organizational resources 'Alice, Bob', implying that they are responsible for securing lodging.
- Name: Archive documents: org::resource; Description: This bind associates the 'Archive documents' activity with the organizational resources 'Alice, Bob', indicating their role in organizing and storing trip-related documents.

Constraints:
- Name: Precedence[Approve application, Apply for Trip B]||B.Application is Application.written; Description: This constraint specifies that the 'Approve application' activity must follow the 'Apply for Trip' activity, provided the trip application is in a 'written' status, ensuring that only documented applications are processed.
- Name: Precedence[Archive documents, Apply for Trip B]||B.Application is Application.written; Description: Indicates that the 'Archive documents' activity can only occur after the 'Apply for Trip' activity if the trip application has been submitted, ensuring proper documentation flow.
- Name: Precedence[Archive documents, Collect tickets B]||B.TicketCollection is TicketCollection.written; Description: Suggests that the 'Archive documents' activity must follow the 'Collect tickets' activity once ticket collection is documented, maintaining order in document processing.
- Name: Precedence[Book means of transport, Approve application]; Description: Ensures that the 'Book means of transport' activity can only be happen after the 'Approve application' activity, aligning travel arrangements with approved plans.
- Name: Precedence[Book accommodation, Approve application]; Description: Ensures that the 'Book accomodation' activity can only happen after the trip 'Approve application', coordinating lodging with approved travel plans.
- Name: Precedence[Collect tickets, Book means of transport]; Description: Ensures that 'Collect tickets' activity follows the 'Book means of transport' activity, streamlining the travel preparation process.
- Name: Precedence[Collect tickets, Book accommodation]; Description: Ensures that 'Collect tickets'follows the 'Book accomodation', facilitating smooth travel arrangements.

- Name: Existence[Apply for Trip]; Description: Ensures that the 'Apply for Trip' activity must occur at least once in the process, initiating the sequence of travel-related activities.
- Name: Existence[Approve application]; Description: Guarantees that the 'Approve application' activity must occur at least once in the process, ensuring all trips are duly authorized.
- Name: Existence[Collect tickets]; Description: Ensures that the 'Collect tickets' activity occurs at least once in the process, securing all necessary tickets for the trip.
- Name: Existence[Archive documents]; Description: Guarantees that the 'Archive documents' activity occurs at least once in the process, ensuring all documents are properly archived post-trip.

# Example 3
- MP-Declare Model:
activity Driving_Test
activity Getting_License
activity Resit
activity Test_Failed

bind Driving_Test: Driver, Grade
bind Getting_License: Driver, Grade
bind Resit: Driver, Grade
bind Test_Failed: Driver

Driver: Fabrizio, Mike, Marlon, Raimundas
Grade: integer between 1 and 5

Response[Driving_Test, Getting_License] | |T.Grade>2 |
Response[Driving_Test, Resit] |A.Grade<=2 | |
Response[Driving_Test, Test_Failed] |A.Grade<=2 | |

- Description:

Activities:
- Name: Driving_Test; Description: The 'Driving_Test' activity represents the initial driving test taken by a candidate.
- Name: Getting_License; Description: The 'Getting_License' activity signifies the action of receiving a driving license upon passing the driving test.
- Name: Resit; Description: The 'Resit' activity represents the action of retaking the driving test after failing.
- Name: Test_Failed; Description: The 'Test_Failed' activity represents the outcome of failing the driving test.


Attributes:
- Name: Driver: Fabrizio, Mike, Marlon, Raimundas; Description: The attribute 'Driver: Fabrizio, Mike, Marlon, Raimundas' represents an individual eligible to participate in the driving test process with possible values being Fabrizio, Mike, Marlon, and Raimundas.
- Name: Grade: integer between 1 and 5; Description: The attribute 'Grade: integer between 1 and 5' is defined as an integer between 1 and 5. It quantifies the performance of a candidate in the driving test, influencing their eligibility for subsequent activities like 'Getting_License' or 'Resit'.

Binds:
- Name: Driving_Test: Driver, Grade; Description: This bind specifies that the 'Driving_Test' activity requires the attributes 'Driver' and 'Grade'. It indicates that each instance of this activity is associated with a specific driver and their corresponding performance grade.
- Name: Getting_License: Driver, Grade; Description: This bind connects the 'Getting_License' activity with the 'Driver' and the 'Grade' they received. It implies that the activity is contingent upon receiving a passing grade, indicating successful completion of the 'Driving_Test'.
- Name: Resit: Driver, Grade; Description: This bind associates the 'Resit' activity with both 'Driver' and 'Grade'. It reflects the need for a driver who failed the initial test to retake it, dependent on their previous grade.
- Name: Test_Failed: Driver; Description: This bind indicates that the 'Test_Failed' activity is solely linked to the 'Driver', without direct consideration of the 'Grade'. It signifies that the activity focuses on the occurrence of failure rather than the specific grade received.

Constraints:
- Name: Response[Driving_Test, Getting_License] | |T.Grade>2 |; Description: This constraint requires that after completing the 'Driving_Test', the 'Getting_License' activity must occur if the candidate's Grade is above 2. It explicitly states that a minimum grade of 3 is necessary to qualify for a license, linking performance directly to licensing outcomes.
- Name: Response[Driving_Test, Resit] |A.Grade<=2 | |; Description: This constraint mandates that candidates who score 2 or below on the 'Driving_Test' must retake the test through the 'Resit' activity. It addresses the potential for candidates to improve and successfully pass on a subsequent attempt.
- Name: Response[Driving_Test, Test_Failed] |A.Grade<=2 | |; Description: This constraint activates the 'Test_Failed' activity if a candidate's grade on the 'Driving_Test' is 2 or below. It formally recognizes and records the failure, potentially leading to a retake of the test.
"""

To structure each construct part (i.e., activities, attributes binds and constraints) description we employed function calling. This time, we separated each construct in a function call and applied a template of generation of description.

$$
\begin{array}{ll}
\textbf{Function} & \textbf{Description} \\
\hline
\text{get_activities_description} & f(\text{activities}) \rightarrow \text{description} \\
\text{get_attributes_description} & f(\text{attributes}) \rightarrow \text{description} \\
\text{get_binds_description} & f(\text{binds}) \rightarrow \text{description} \\
\text{get_constraints_description} & f(\text{constraints}) \rightarrow \text{description} \\
\hline
\end{array}
$$

$$
\begin{aligned}
\text{where} \\
f(x) &: \text{Apply template to each } x_i \in x \rightarrow \text{generated description} \\
x &\in \{\text{activities}, \text{attributes}, \text{binds}, \text{constraints}\} \\
x_i &: \{ \text{name} : \text{string}, \text{description} : \text{string} \} \\
\text{Required Properties:} \\
\text{name} &: \text{The name of the } x_i \\
\text{description} &: \text{The description of the } x_i \\
\end{aligned}
$$


In [ ]:
# 4 functions to get model description.
toolsdescription = [
  {
    "type": "function",
    "function": {
      "name": "get_activities_description",
      "description": "Fetch the activities of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "activities": {
            "type": "array",
            "description": "The activities contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the activity."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the activity."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_attributes_description",
      "description": "Fetch the attributes of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "attributes": {
            "type": "array",
            "description": "The attributes contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the attribute."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the attribute."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_binds_description",
      "description": "Fetch the binds of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "binds": {
            "type": "array",
            "description": "The binds contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the bind."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the bind."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_constraints_description",
      "description": "Fetch the constraints of a given MP-Declare model and generate their descriptions following the template.",
      "parameters": {
        "type": "object",
        "properties": {
          "constraints": {
            "type": "array",
            "description": "The constraints contained in the model followed by generated descriptions.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the constraint."
                },
                "description": {
                  "type": "string",
                  "description": "The description of the constraint."
                }
              },
              "required": ["mp_declare_model", "format", "string"]
            }
          }
        }
      }
    }
  }
]

In [ ]:
model ="\n".join(declare_model.declare_model_lines)
model = "New model to generate description of activities, attributes, binds and constraints based in the template: \n" + model
print(model)

In [ ]:
# Run prompt to generate intermediary description
messagesmptodesc=[
            {
                "role": "system",
                "content": decltotextassistant
            },
            {
                "role": "user",
                "content": model
            }
        ]

response_decltointermediary = chat_completion_request(
        messages=messagesmptodesc,
        tools= toolsdescription,
        tool_choice="auto",
        temperature=0.2
        )

In [ ]:
tool_call = response_decltointermediary.choices[0].message.tool_calls
function_arguments = tool_call

In [ ]:
def process_intermediate_descriptions(data):
    model = {
        "Activities": [],
        "Attributes": [],
        "Binds": [],
        "Constraints": []
    }

    for item in data:
        func_args = json.loads(item.function.arguments)
        for key, value in func_args.items():
            for v in value:
                model[key.capitalize()].append(f"{v['name']}: {v['description']}")

    model_string = ""
    for key, descriptions in model.items():
        model_string += f"{key}:\n"
        for desc in descriptions:
            model_string += f"  - {desc}\n"

    return model_string

description_string = process_intermediate_descriptions(response_decltointermediary.choices[0].message.tool_calls)
print(description_string)


##Enhanced Description Generation

This is the final step of our solution. It will take as input the intermediary descriptions and outputs the complete description of the MP-Declare model. It is necessary to mention that this entire description entangles all aspects of model constructs and how they interplay with each other.

In order to generate the description, the model will take a prompt with instructs in format of role-play and the intermediary descriptions of the process.

In [ ]:
enhdescassistant ="""
# AI Business Process Interpretation Prompt

You are an AI trained to interpret and describe business processes based on declarative process models. Your task is to generate a continuous and detailed description of business processes from a set of provided activities, attributes, binds, and constraints. Follow this strategy when describing the process:

## 1. Overall Activity Context

- Describe the general purpose and scope of the process.
- Explain how activities are generally related to each other.

## 2. Process Flow Description

Begin with the process initiation and progress through each activity, ensuring a logical flow that reflects the constraints and relationships defined in the model. For each activity:

- Explain how it connects to previous and subsequent activities.
- Highlight attributes influencing its execution.
- Detail how specific constraints control the flow to and from this activity.
- Consider dependencies, order of execution, and the impact of outcomes from previous activities.

## 3. Constraint Integration

Throughout the description, incorporate the following constraints only if they apply to activities in the original model:

### Unary Constraints

- **Existence**: Specify when an activity must occur.
- **Absence**: Indicate when an activity must not occur.
- **Exactly**: Describe when an activity must occur a specific number of times.
- **Init**: Identify activities that can or must start the process.
- **End**: Point out activities that can or must end the process.

### Binary Constraints

- **Choice and Exclusive Choice**: Explain options between activities.
- **Responded Existence**: Describe when the occurrence of one activity requires another.
- **Response, Alternate Response, and Chain Response**: Detail how activities trigger others.
- **Precedence, Alternate Precedence, and Chain Precedence**: Clarify required order of activities.
- **Succession, Alternate Succession, and Chain Succession**: Explain bidirectional dependencies.
- **Co-Existence**: Highlight activities that must occur together.
- **Not Chain Succession, Not Co-Existence, Not Succession**: Describe prohibited sequences or combinations.
- **Not Responded Existence, Not Response, Not Precedence**: Explain when certain follow-ups are forbidden.
- **Not Chain Response, Not Chain Precedence**: Detail prohibited immediate sequences.

## 4. Constraint Application

For each constraint applied:

- Explain its impact on the process flow.
- Describe how it affects decision-making or branching in the process.
- Clarify any conditions or exceptions related to the constraint.

## 5. Conclusion

Conclude by summarizing the key aspects of the process, highlighting the most critical constraints and their overall impact on the business process execution.

---

**Note**: Maintain a logical flow in your description, connecting activities and constraints in a way that reflects real-world process execution. Your explanation should be detailed yet comprehensible, suitable for both process experts and those new to the specific business domain.\n""" + description_string

In [ ]:
# Run prompt to generate Enhancend Descriptions
messagesendesc=[
            {
                "role": "system",
                "content": enhdescassistant
            },
            {
                "role": "user",
                "content": "New model to generate description: " + description_string
            }
        ]

responseendesc = chat_completion_request(
        messages=messagesendesc,
        temperature=0.2
        )

In [ ]:
enhanced_description_string = responseendesc.choices[0].message.content
print(enhanced_description_string)

##Evaluation Functions

In [ ]:
# Functions to evaluate MP-Declare model descriptions
toolsjudge = [
  {
    "type": "function",
    "function": {
      "name": "evaluate_guideline_adherence",
      "description": "Evaluate how well the description adheres to given guidelines.",
      "parameters": {
        "type": "object",
        "properties": {
          "rating": {
            "type": "integer",
            "description": "Rating from 1-10 on how well the description adheres to the given guidelines."
          },
          "justification": {
            "type": "string",
            "description": "Justification for the guideline adherence rating."
          }
        },
        "required": ["mp_declare_model_and_description", "guidelines"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "evaluate_essential_elements_capture",
      "description": "Evaluate how well the description captures the essential elements of the MP-Declare model.",
      "parameters": {
        "type": "object",
        "properties": {
          "rating": {
            "type": "integer",
            "description": "Rating from 1-10 on how well the description captures essential elements of the MP-Declare model."
          },
          "justification": {
            "type": "string",
            "description": "Justification for the essential elements capture rating."
          }
        },
        "required": ["mp_declare_model_and_description"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "evaluate_non_technical_understanding",
      "description": "Evaluate how well the description facilitates understanding for someone without a technical background in process modeling.",
      "parameters": {
        "type": "object",
        "properties": {
          "rating": {
            "type": "integer",
            "description": "Rating from 1-10 on how well the description facilitates understanding for non-technical audiences."
          },
          "justification": {
            "type": "string",
            "description": "Justification for the non-technical understanding rating."
          }
        },
        "required": ["mp_declare_model_and_description"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "evaluate_detail_clarity_balance",
      "description": "Evaluate how well the description balances providing detailed information with maintaining overall clarity.",
      "parameters": {
        "type": "object",
        "properties": {
          "rating": {
            "type": "integer",
            "description": "Rating from 1-10 on how well the description balances detail and clarity."
          },
          "justification": {
            "type": "string",
            "description": "Justification for the detail-clarity balance rating."
          }
        },
        "required": ["mp_declare_model_and_description"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "evaluate_data_resource_explanation",
      "description": "Evaluate the description's effectiveness in explaining how data attributes and resources are integrated into the process model.",
      "parameters": {
        "type": "object",
        "properties": {
          "rating": {
            "type": "integer",
            "description": "Rating from 1-10 on the effectiveness of explaining data attributes and resource integration."
          },
          "justification": {
            "type": "string",
            "description": "Justification for the data and resource explanation rating."
          }
        },
        "required": ["mp_declare_model_and_description"]
      }
    }
  }
]

In [ ]:
judgeassistant=f"""# LLM-as-a-Judge Prompt for Evaluating MP-Declare Process Descriptions

You are an expert judge tasked with evaluating process descriptions generated from MP-Declare models. Your role is to provide an impartial and thorough assessment of the description's quality and effectiveness. Please carefully analyze the given MP-Declare model and final description and answer the following questions:

## Evaluation Questions

1. **Adherence to Guidelines**
   Based on the following guidelines:
   {enhdescassistant}
   Rate how well the final description adheres to these guidelines on a scale of 1-10.
   Provide a detailed justification for your rating, citing specific examples from the description.

2. **Capturing Essential Elements**
   On a scale of 1-10, how well does the generated final description capture the essential elements of the MP-Declare model?
   Explain your rating by discussing which elements are well-represented and which, if any, are missing or unclear.

3. **Facilitating Understanding for Non-Experts**
   On a scale of 1-10, how well does the final description facilitate understanding of the process model for someone without a technical background in process modeling?
   Justify your rating by considering the clarity of explanations, use of jargon, and overall accessibility of the content.

4. **Balance of Detail and Clarity**
   On a scale of 1-10, how well does the final description balance providing detailed information with maintaining overall clarity?
   Support your rating with examples of where this balance is achieved or could be improved.

5. **Explaining Data Attributes and Resources**
   Rate from 1-10 the final description's effectiveness in explaining how data attributes and resources are integrated into the process model.
   Provide a detailed justification for your rating, highlighting strengths and areas for improvement.

## Important Instructions

- Before providing your ratings, carefully read through the entire description at least twice to ensure a comprehensive understanding.
- For each question, first provide your numerical rating, then offer a detailed justification.
- In your justifications, refer to specific parts of the description to support your evaluation.
- Consider the hierarchy of constraints in the MP-Declare model when evaluating the description's structure and clarity.
- Assess the description's coherence and flow, ensuring it presents information in a logical and understandable sequence.
- Pay attention to how well the description explains the relationships between different elements of the process model.

Your evaluation should be thorough, impartial, and constructive. Provide specific suggestions for improvement where applicable."""

In [ ]:
judgeprompt=f"""The MP-Declare model\n: {description_string} \n The final description:\n {enhanced_description_string}\n"""

In [ ]:
print(judgeprompt)

In [ ]:
# Run prompt to generate benchmark
messagesjudge=[
            {
                "role": "system",
                "content": judgeassistant
            },
            {
                "role": "user",
                "content": judgeprompt
            }
        ]

response_judge = chat_completion_request(
        messages=messagesjudge,
        tools= toolsjudge,
        tool_choice="auto",
        temperature=0.2
        )

In [ ]:
tool_call = response_judge.choices[0].message.tool_calls
function_arguments = tool_call

In [ ]:
print(function_arguments)

In [ ]:

def process_tool_calls(tool_calls: List['ChatCompletionMessageToolCall']) -> dict:
    result = {}
    for call in tool_calls:
        function_name = call.function.name
        arguments = json.loads(call.function.arguments)

        result[function_name] = {
            'rating': arguments['rating'],
            'justification': arguments['justification']
        }

    return result

# Example usage
processed_data = process_tool_calls(function_arguments)
print(json.dumps(processed_data, indent=2))